In [ ]:
import math
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import gurobipy as gp
import warnings
#不显示警告信息
warnings.filterwarnings("ignore")


### 集合\参数设置

#### 一天的时间片序列【5min、15min】

In [ ]:
#时刻分配的5时间片序列
T5=[i for i in range(288)]
#容量的15时间片序列
T15=[i for i in range(96)]


#### 设置航季的天数索引 $$2019-03-31==0$$ <br> $$2019-10-26==209$$

In [ ]:
#2019夏航季的210天
D=[i for i in range(210)]


#### 51个日均流量大于等于100的机场

In [ ]:
#机场列表【这是按照日均航班顺序排列的，也可以是其他顺序，不影响】
A= ['ZGGG', 'ZSPD', 'ZGSZ', 'ZBAA', 'ZUTF', 'ZPPP', 'ZLXY', 'ZUCK', 'ZSHC', 'ZBAD', 'ZSSS', 'ZSNJ', 'ZUUU', 'ZHHH', 'ZGHA', 'ZHCC', 'ZSAM', 'ZWWW', 'ZSQD', 'ZJHK', 'ZYTX', 'ZBTJ', 'ZYHB', 'ZUGY', 'ZSJN', 'ZYTL', 'ZJSY', 'ZLLL', 'ZYCC', 'ZGNN', 'ZBHH', 'ZBYN', 'ZSNB', 'ZSOF', 'ZSFZ', 'ZSWZ', 'ZSCN', 'ZGSD', 'ZSWX', 'ZBSJ', 'ZSYT', 'ZLXN', 'ZLIC', 'ZPLJ', 'ZGOW', 'ZPJH', 'ZSQZ', 'ZGKL', 'ZULS', 'ZSCG', 'ZSNT']


#### 容量情景导入

In [ ]:
zdep=r'../data with adjustment/1.two-stage model and NS-DSC/DEP'
zarr=r'../data with adjustment/1.two-stage model and NS-DSC/ARR'
L=[[],[]]#L[0]为data_dep，L[1]为data_arr，存储初始的210个dep、arr容量数据
#导入def的输入
def read_data(zdep,zarr,L):
    #得到地址zdep和zarr下的csv文件列表
    list_dep = os.listdir(zdep)
    list_arr = os.listdir(zarr)
    ADD=[list_dep,list_arr]
    fumulu=[zdep,zarr]
    #将csv文件列表中的csv文件读取为dataframe
    for j,i,h in zip(L,ADD,fumulu):#j为data_dep或data_arr，i为list_dep或list_arr,j和i一一对应,即data_dep对应list_dep,data_arr对应list_arr
        for k in range(len(i)):
            j.append(pd.read_csv(os.path.join(h, i[k])).iloc[:, 0:])
    return L
L=read_data(zdep,zarr,L)

l=[0,1,2]
#将Unnamed: 0列的数据换为A列表中的机场名
for i in [0,1]:
    for j in l:
        L[i][j]['Unnamed: 0']=A

S=[[],[]]#S[0]为data_dep，S[1]为data_arr，存储初始的4个情景的45个机场的dep、arr容量数据
add=['-D','-A']
#S保留L中各元素内的l个元素
for i in l:
    for j,k in zip([0,1],add):
        #将第0列列名Unnamed:0改为airport
        L[j][i].rename(columns={'Unnamed: 0':'airport'},inplace=True)
        L[j][i]['airport']=L[j][i]['airport']+k
        #
        S[j].append(L[j][i])

#S[0]为data_dep，S[1]为data_arr，存储初始的3个情景的45个机场的dep、arr容量数据
#S[0][0]为第一个情景的dep容量数据，S[1][0]为第一个情景的arr容量数据

qingjing=[0,1,2]
qingjing_weight=[0.25, 0.32142857142857145, 0.42857142857142855]


#### 输入input1，这是原始航班时刻数据。其中有错误数据，需要处理

In [ ]:
input1=pd.read_csv(r'../data with adjustment/1.two-stage model and NS-DSC/input1.csv')#dep/arr拆分后的航班申请序列，总共103476个
#删除Unnamed:0列
input1.drop(columns='Unnamed: 0',inplace=True)
#将shikexulie的shibiehao列等字符串列表转换为数值列表
for i in range(input1.shape[0]):
    for j in ['shibiehao','TIAN_DEP','TIAN_ARR']:
        input1[j][i]=eval(input1[j][i])


In [ ]:
input1=input1[~input1['xuhao'].isin([1227,12847,13436,59477,59514,26499, 6669, 39316, 41880, 4761, 37149, 52527, 26931, 8131, 28359, 3533, 51024, 54865, 10839, 12643, 28388, 31974, 6379, 496, 33650, 25213,26118, 27142, 26120, 21511, 24590, 1039, 2580, 60948, 12312, 24603, 6685, 9254, 25127, 4648, 60972, 22066, 51, 54326, 60986, 20539, 14396, 1597, 11326, 7228, 3137, 28739, 5189, 5714, 14934, 43606, 7260, 28253, 3681, 27236, 28260, 61031, 19559, 22639, 14959, 18034, 1650, 61043, 39032, 3705, 39034, 10875, 11908, 6789, 6795, 28817, 24725, 10908, 6308, 1700, 61093, 17064, 12974, 57518, 26803, 30392, 2233, 2232, 12475, 8897, 5825, 4807, 12488, 10440, 47816, 16594, 2262, 28887, 8414, 13535, 3301, 3820, 27885, 8940, 3825, 39158, 5880, 55546, 25339, 55550, 17151, 12542, 263, 27403, 28428, 2829, 43277, 27405, 14097, 37655, 31512, 5401, 25883, 9500, 4381, 6434, 4387, 55078, 19751, 25387, 28460, 25900, 14643, 2362, 2369, 16194, 24392, 9545, 58698, 19786, 12621, 13646, 2383, 5456, 8014, 24912, 3414, 12120, 26461, 20321, 20835, 11107, 2405, 26471, 24936, 1385, 6505, 36717, 7023, 27507, 39283, 46453, 24952, 25465, 6011, 18299, 21372, 8062, 28031, 24957, 10113, 29058, 1924, 27015, 393, 8073, 24977, 13714, 14744, 30617, 28572, 6047, 9121, 13217, 60834, 1447, 28591, 24496, 4531, 5053, 20926, 47552, 52676, 2501, 26567, 4554, 14283, 14794, 460, 19918, 6095, 38352, 8660, 42968, 2521, 8153, 15320, 473, 30682, 27096, 1504, 28640, 28643, 25059, 25062, 60903, 16878, 7152, 60925, 25599])]
input1=input1.reset_index(drop=True)

#### 提取2023.7.29-2023.7.31的航班input【总共41696个航班系列，航班量为91070，考虑3种运行情景后即为273210】

In [ ]:
#得到input1每一行TIAN_DEP列值的最大值，如果小于150，就将其索引值加入index列表
index=[]
for i in range(input1.shape[0]):
    if max(input1['TIAN_DEP'][i])<150:
        index.append(i)

#将这些索引值对应的行删除
input1.drop(index,inplace=True)
input1=input1.reset_index(drop=True)
print(input1.shape)

for i in input1.index.tolist():
    #将input1的TIAN_DEP列值中小于150的元素删去
    input1['TIAN_DEP'][i]=[j for j in input1['TIAN_DEP'][i] if j>=150]
    #将input1的TIAN_ARR列值中小于150的元素删去
    input1['TIAN_ARR'][i]=[j for j in input1['TIAN_ARR'][i] if j>=150]
    len_dep=len(input1['TIAN_DEP'][i])
    len_arr=len(input1['TIAN_ARR'][i])
    if len_dep!=len_arr:
        #将input1的TIAN_ARR列值的第一个元素删去
        input1['TIAN_ARR'][i]=input1['TIAN_ARR'][i][1:]
        len_arr1=len(input1['TIAN_ARR'][i])
        if len_dep!=len_arr1:
            print('删除失败{}'.format(i))
    input1.loc[i,'day_len']=len_dep
input1=input1.reset_index(drop=True)

sum=0
for i in input1.index.tolist():
    sum+=input1['day_len'][i]
print('这周的航班量为{}'.format(sum))


In [ ]:
#依据申请天数为区分的分组函数
def group_by_daylen(input1):
    divide_day_len=dict()
    daylen=set(input1['day_len'])
    for i in daylen:
        #重置索引
        divide_day_len[i]=input1[input1['day_len']==i].reset_index(drop=True)
    return divide_day_len

divide_day_len=group_by_daylen(input1)

def lianchenshengcheng(divide_day_len):
    Z=[]
    #依此循环每个key对应的value
    key=list(divide_day_len.keys())
    value=list(divide_day_len.values())
    for z in range(len(value)):
        a=value[z]['arraddr'].unique()
        b=value[z]['depaddr'].unique()
        #a和b的交集
        c=list(set(a).intersection(set(b)))
        for y in c:
            #得到arraddr为UNNT且dep0/arr1为1的行
            d=value[z][(value[z]['arraddr']==y)&(value[z]['dep0/arr1']==1)]#y机场的降落航班
            e=value[z][(value[z]['depaddr']==y)&(value[z]['dep0/arr1']==0)]#y机场的起飞航班
            f=d['TIAN_ARR'].tolist()#得到预备第一程中TIAN_ARR的值
            g=e['TIAN_DEP'].tolist()#得到预备第二程中TIAN_DEP的值

            h=[]#得到f和g中相同的元素，即为第一程TIAN_ARR和第二程TIAN_DEP相同的元素
            for i in f:
                if i in g:
                    h.append(i)

            #如果不为空，表面有运行天相同的同机场y的降落和起飞航班
            if len(h)!=0:
                #用元组进行去重
                h = [list(item) for item in set(tuple(item) for item in h)]
                #print(h)
                #得到相同运行天集合后，进一步考虑航班第一程到达时间和航班第二程出发时间的关系
                for i in h:
                    i=[i]#将i转换为列表,很奇怪，如果不转换为列表，下面的代码会报错，是因为i是一个元组，而不是列表
                    #得到d中TIAN_ARR为i的行的aslot的值
                    aa=d[d['TIAN_ARR'].isin(i)]['aslot'].tolist()
                    bb=e[e['TIAN_DEP'].isin(i)]['dslot'].tolist()
                    cc=d[d['TIAN_ARR'].isin(i)]['xuhao-dep0/arr1'].tolist()#得到第一程的航班号
                    dd=e[e['TIAN_DEP'].isin(i)]['xuhao-dep0/arr1'].tolist()#得到第二程的航班号
                    #print(aa,bb,cc,dd)
                    #判断h中每个同天元素的第一程到达时间和第二程出发时间的关系
                    for j in range(len(aa)):
                        for k in range(len(bb)):
                            #设置一个阈值，第一程到达时间和第二程出发时间的差值在12个5min时间片之间
                            if aa[j]<=bb[k]-12 and aa[j]>=bb[k]-12:#【都是一串联程航班均处于同一天内，可以多次联程】
                                #得到cc中j的xuhao-dep0/arr1列的值
                                ee=cc[j]#得到第一程的航班号
                                ff=dd[k]#得到第二程的航班号
                                value[z].loc[(value[z]['xuhao-dep0/arr1']==ee),'liancheng']=str([ee,ff])
                                value[z].loc[(value[z]['xuhao-dep0/arr1']==ff),'liancheng']=str([ee,ff])
                                #将bb的第k个元素设置为-1，表示已经被使用，确保不会重复使用，因为一个航班只能有一个联程航班
                                bb[k]=-1
                                break#跳出最近的一层循环，而非跳出if，if不是循环
            else:
                pass
        #如果value[z]中没有liancheng列，则添加此列，赋值为NaN
        if 'liancheng' not in value[z].columns:
            value[z]['liancheng']=np.nan
            Z.append(z+1)
    #print(Z)        
    return divide_day_len

liancheng=lianchenshengcheng(divide_day_len)

# %%
#从liancheng中得到联程航班的数据，进行再分组【分组函数，带有联程航班】
def group_by_daylen_liancheng(liancheng):    
    divide_day_len_liancheng=dict()
    divide_day_len_putong=dict()
    #依此循环每个key对应的value
    key=list(liancheng.keys())
    value=list(liancheng.values())
    for i in range(len(value)):
        #得到value[i]中的lianchen列不为空的行的xuhao列的值
        lianchenghangbanxuhao=value[i][value[i]['liancheng'].notnull()]['xuhao'].tolist()
        #删去重复值
        lianchenghangbanxuhao=list(set(lianchenghangbanxuhao))
        divide_day_len_liancheng[i+1]=value[i][value[i]['xuhao'].isin(lianchenghangbanxuhao)]
        divide_day_len_putong[i+1]=value[i][~value[i]['xuhao'].isin(lianchenghangbanxuhao)]
    return divide_day_len_liancheng,divide_day_len_putong

input_liancheng,input_putong=group_by_daylen_liancheng(liancheng)

for i in range(1,4):#判断数据长度是否正确
    if len(input_liancheng[i])+len(input_putong[i])!=len(liancheng[i]):
        print('{}错误'.format(i))


In [ ]:
#为各个决策变量添加时间范围
def shengchengjuecebianliangshike(input_liancheng,input_putong):    
    for k in [input_liancheng,input_putong]:
        for j in range(1,4):
            a=k[j]
            #为a添加1列t1dep和tⅠarr，分别表示每个航班战略阶段dep/arr的时间区间
            #dslot-12到dslot+12的列表为t1dep，aslot-12到aslot+12的列表为t1arr
            #得到a的所有索引值
            index=a.index.tolist()
            for i in index:
                #得到i对应的dslot和aslot的值
                dslot,aslot=a.loc[i,'dslot'],a.loc[i,'aslot']
                aa,bb,cc,dd=dslot-3,dslot+3,aslot-3,aslot+3#战略时刻变化值设置
                if aa<0:
                    aa=0
                if bb>287:
                    bb=287
                if cc<0:
                    cc=0
                if dd>287:
                    dd=287
                #战略延误值的最大值设置
                a.loc[i,'t1dep'],a.loc[i,'t1arr'] = str(list(range(int(aa), int(bb)+1))),str(list(range(int(cc), int(dd)+1)))
                
                #这是为了得到每个航班的飞行时间的最小值和最大值
                flytime=a.loc[i,'flytime']
                ee,ff=int(flytime*0.9),int(flytime*1.1)+1#空中飞行时间min与max设置
                a.loc[i,'minflytime'],a.loc[i,'maxflytime']=ee,ff
                
                #战术延误的最大值设置为36个时间片
                gg,hh,ii,jj=int(aa),int(bb+36),int(aa+ee),int(dd+36)
                if gg<0:
                    gg=0
                if hh>287:
                    hh=287
                if ii<0:
                    ii=0
                if a.loc[i,'TIAN_DEP']!=a.loc[i,'TIAN_ARR']:
                    if ii>287:#避免跨天时间不对，input_putong会有跨天的情况出现，所以要加上这个判断
                        ii=ii-288
                    else:
                        ii=0
                if jj>287:
                    jj=287
                #战术延误的最大值设置
                a.loc[i,'t2dep'],a.loc[i,'t2arr'] = str(list(range(gg, hh+1))),str(list(range(ii, jj+1)))
                
                #为a添加一列T_wit，如果这行的dep0/arr1列的值为0，则该行的T_wit列值为t1dep列值
                if a.loc[i, 'dep0/arr1'] == 0:#起飞航班
                    a.loc[i, 'Tw_it'] = str(a.loc[i, 't1dep'])
                    a.loc[i, 'Tx_itds'] = str(a.loc[i, 't2dep'])
                    a.loc[i, 'x_TIAN'] = str(a.loc[i, 'TIAN_DEP'])
                    a.loc[i,'Tfp']=a.loc[i,'dslot']
                else:
                    a.loc[i,'Tw_it']=str(a.loc[i,'t1arr'])
                    a.loc[i,'Tx_itds']=str(a.loc[i,'t2arr'])
                    a.loc[i, 'x_TIAN'] = str(a.loc[i, 'TIAN_ARR'])
                    a.loc[i,'Tfp']=a.loc[i,'aslot']
    return input_liancheng,input_putong

input_liancheng,input_putong=shengchengjuecebianliangshike(input_liancheng,input_putong)
#copy使两者互不影响
input_liancheng1,input_putong1=input_liancheng.copy(),input_putong.copy()


In [ ]:
#判断航班时刻表数据合理性
def delete_abnormal(input_liancheng1,input_putong1):
    delete=[]
    for g in [input_liancheng1,input_putong1]:    
        #判断跨天航班的决策变量起降时刻设置是否有问题
        for i in range(1,4):
            #如果input_liancheng1[i]为空，则跳过
            if g[i].shape[0]==0:
                continue
            #判断input_liancheng1是否有单元格为[]
            else:
                for j in ['t1dep','t1arr','t2dep','t2arr','Tw_it','Tx_itds','x_TIAN']:
                    if g[i][g[i][j]=='[]'].shape[0]!=0:
                        print('错误{}'.format(i))
            #判断TIAN_DEP和TIAN_ARR是否相同
                for k in g[i].index.tolist():
                    if g[i]['TIAN_DEP'][k]!=g[i]['TIAN_ARR'][k]:
                        #如果不同则判断TIAN_ARR的所有元素减1是否等于TIAN_DEP
                        if (np.array(g[i]['TIAN_ARR'][k])-1==np.array(g[i]['TIAN_DEP'][k])).all():
                            pass
                        else:
                            #print('错误{}'.format(i))
                            #print(g[i].loc[k])
                            #删除错误的行,k是行索引，dep/arr会同时出现天数错误，会一块删除dep/arr
                            g[i].drop(k,inplace=True)
                            #delete.append(g[i].loc[k,'liancheng'])
    return input_liancheng1,input_putong1#,delete

input_liancheng1,input_putong1=delete_abnormal(input_liancheng1,input_putong1)


### 规模估算、再拆分字典的建立

In [ ]:
#计算input_liancheng1和input_putong1的航班个数
def jisuanhangbangeshu(input_liancheng1,input_putong1):
    AA,BB=0,0
    for i in range(1,4):
        AA+=input_liancheng1[i].shape[0]*i
        BB+=input_putong1[i].shape[0]*i
    return AA,BB
AA,BB=jisuanhangbangeshu(input_liancheng1,input_putong1)
print(AA,BB,AA+BB)


#### 建立联程航班的序号依赖字典【用于再拆分和第二步的航班优化筛选】

In [ ]:
#设置联程航班的最大相关航班字典
def liancheng_yilai(input_liancheng1):
    liancheng_xinxi=[]
    for i in range(1,4):
        #将input_liancheng1[i]中的liancheng列值提取出来，存入liancheng_xinxi列表中
        for j in input_liancheng1[i].index.tolist():
            liancheng_xinxi.append(input_liancheng1[i]['liancheng'][j])
    #删除重复值
    liancheng_xinxi=list(set(liancheng_xinxi))

    #将liancheng_xinxi中浮点数删除，这样就把各种不同的nan删除了，它的类型为float，不是str
    liancheng_xinxi=[i for i in liancheng_xinxi if type(i)!=float]
    #print(liancheng_xinxi)

    liancheng_xinxishuju=[]
    for i in range(len(liancheng_xinxi)):
        liancheng_xinxishuju.append([int(liancheng_xinxi[i].split(',')[0][1:-1].replace("'",'')[0:-2]),int(liancheng_xinxi[i].split(',')[1][1:-1].replace("'",'')[0:-2])])
    #print(liancheng_xinxishuju)
    # #检验过没有重复值，没有循环联程出现

    xuhao_setlist=[]
    for i in range(len(liancheng_xinxi)):
        for j in liancheng_xinxishuju[i]:
            xuhao_setlist.append(j)
    xuhao_setlist=list(set(xuhao_setlist))#由9962变为9129，说明有833个多段联程航班
    #print(xuhao_setlist)

    def quchong(l):
        l=list(set(l))
        return l

    #创造xuhao_setlist中联程航班xuhao的最大相关航班字典
    dict_max_relation_xuhao={}
    for i in xuhao_setlist:#xuhao_setlist:
        #将liancheng_xinxishuju中的元素包含i的元素列表提取出来
        l=[]
        a=[j for j in liancheng_xinxishuju if i in j]
        #print(a)
        l=quchong([i for j in a for i in j])
        #rint(l)

        l2=[]
        a=[j for i in range(len(l)) for j in liancheng_xinxishuju if l[i] in j]
        #print(a)
        l2=quchong([i for j in a for i in j])
        #print(l2)

        while set(l)!=set(l2):#如果l和l2不相等,则继续循环,否则跳出循环
            l=l2.copy()
            a=[j for i in range(len(l2)) for j in liancheng_xinxishuju if l2[i] in j]
            #print(a)
            l2=quchong([i for j in a for i in j])
            #print(l2)
        dict_max_relation_xuhao[i]=l2
    return dict_max_relation_xuhao

dict_max_relation_xuhao=liancheng_yilai(input_liancheng1)


#### 计算需要再分组的航班申请

In [ ]:
#计算需要再分组的已分组航班申请
def zaichaifen(input_liancheng1,input_putong1):
    import math
    dict_zaichaifen={}
    for i,k in zip([input_liancheng1,input_putong1],['liancheng','putong']):#56、210个非空
        number_dancihangban=0
        input_11=i.copy()
        # 对于input_liancheng11判断有多少字典的键值对的value为空
        empty_11=[i for i in input_11 if input_11[i].shape[0]==0]
        input_11={i:input_11[i] for i in input_11 if i not in empty_11}
        #每个键对应的值中包含的行数【此时航班已经拆分为dep/arr】
        dict_11={i:input_11[i].shape[0] for i in input_11}#这里的i不会受到上面的i的影响，因为这里的i是一个局部变量
        #将dict_11的key和value相乘，作为新的value值
        dict_12 = {i: i * dict_11[i] for i in dict_11.keys()}
        #print(dict_12)
        #设置要分组的信息dict
        dict_13= {i: math.ceil(dict_12[i]/1000) for i in dict_12.keys() if math.ceil(dict_12[i]/1000)!=1}
        dict_zaichaifen[k]=dict_13
    return dict_zaichaifen

zaichaifen_dict=zaichaifen(input_liancheng1,input_putong1)
zaichaifen_dict


#### 联程航班申请的再拆分

In [ ]:
#联程航班再拆分函数得到lc_input
def lc_zaichaifen(input_liancheng1,zaichaifen_dict,dict_max_relation_xuhao):
    lc=input_liancheng1.copy()
    lc_input={}
    for i in range(1,4):
        lc_input[i]={}#对于每一个i，都创建一个空字典，用来存放i天的再拆分得到的航班申请数据

    for i in range(1,4):#1天、2天、3天这3种联程形式
        if lc[i].shape[0]==0:#如果input_liancheng1[i]为空，则删除【符合该天数的航班为空】
            del lc_input[i]
        elif i not in list(zaichaifen_dict['liancheng'].keys()):
            lc_input[i][0]=lc[i]
        else:
            nn=zaichaifen_dict['liancheng'][i]#nn为再拆分的分组数
            NN=lc[i].shape[0]//nn#每组的行数，最后一组的行数不做要求
            list_xuhao=list(set(lc[i]['xuhao']))#得到i天的所有航班申请序列的xuhao列值，这些是待再拆分的航班xuhao
            list_xuhao.sort()
            
            chaifenxuhao={}#存放再拆分的航班xuhao
            for j in range(0,zaichaifen_dict['liancheng'][i]):#这里是再拆分的组数
                l_xuhao=[]
                while len(l_xuhao)*2<NN and len(list_xuhao)!=0:
                    #得到list_xuhao中的第1个元素在dict_max_relation_xuhao中的value值，本次要添加的所有相关联程航班
                    l_add=dict_max_relation_xuhao[list_xuhao[0]]#本次的相关联程航班
                    for k in l_add:
                        #if k in list_xuhao:
                        list_xuhao.remove(k)
                    l_xuhao=l_xuhao+l_add
                chaifenxuhao[j]=l_xuhao
            #如果list_xuhao中还有元素，则打印‘还有元素’
            if len(list_xuhao)!=0:
                print('还有元素{}'.format(i))
                print(list_xuhao)

                
            #将chaifenxuhao中的元素添加到lc_input[i]中
            for j in chaifenxuhao:
                lc_input[i][j]=lc[i][lc[i]['xuhao'].isin(chaifenxuhao[j])]
            
            #检验是否都被提取出来
            #计算所有chaifenxuhao中的元素个数和
            sum=0
            for j in chaifenxuhao:
                #print(len(chaifenxuhao[j]))
                sum+=len(chaifenxuhao[j])
            #print(sum)
            if sum!=len(lc[i]['xuhao'])/2:
                print('错误{}'.format(i))
            
            SUM=0
            for j in chaifenxuhao:
                #print(lc_input[i][j].shape[0])
                SUM+=lc_input[i][j].shape[0]
            #print(SUM)
            if SUM!=lc[i].shape[0]:
                print('错误{}'.format(i))
            
    #删除lc_input中的空字典
    for i in list(lc_input.keys()):
        for j in list(lc_input[i].keys()):
            if lc_input[i][j].shape[0]==0:
                del lc_input[i][j]
                #print('删除{}-{}'.format(i,j))
    return lc_input

lc_input=lc_zaichaifen(input_liancheng1,zaichaifen_dict,dict_max_relation_xuhao)
lc_input1=lc_zaichaifen(input_liancheng1,zaichaifen_dict,dict_max_relation_xuhao)
lc_input2=lc_zaichaifen(input_liancheng1,zaichaifen_dict,dict_max_relation_xuhao)
#判断lc_input1和lc_input2这两个字典中的每个键对应的值是否相等
lc_zushu=0
lc_geshu=0
for i in list(lc_input1.keys()):
    for j in list(lc_input1[i].keys()):
        lc_zushu+=1
        lc_geshu+=lc_input1[i][j].shape[0]*i
        if all(lc_input1[i][j] == lc_input2[i][j]):
            pass
        else:
            print('错误{}'.format(i))
print(lc_zushu,lc_geshu)


#### 普通航班申请的再拆分

In [ ]:
#普通航班再拆分函数得到pt_input
def pt_zaichaifen(input_putong1,zaichaifen_dict):
    pt=input_putong1.copy()
    pt_input={}
    for i in range(1,4):
        pt_input[i]={}#对于每一个i，都创建一个空字典，用来存放i天的再拆分得到的航班申请数据

    for i in range(1,4):
        if pt[i].shape[0]==0:#如果pt[i]为空，则删除
            del pt_input[i]
        elif i not in list(zaichaifen_dict['putong'].keys()):#如果i不在再拆分的字典中，则不再拆分
            pt_input[i][0]=pt[i]
        else:
            nn=zaichaifen_dict['putong'][i]#nn为再拆分的分组数
            NN=pt[i].shape[0]//nn#每组的行数，最后一组的行数不做要求
            list_xuhao=list(set(pt[i]['xuhao']))#得到i天的所有航班申请序列的xuhao列值，这些是待再拆分的航班xuhao
            list_xuhao.sort()
            
            chaifenxuhao={}
            for j in range(0,zaichaifen_dict['putong'][i]+1):
                l_xuhao=[]
                while len(l_xuhao)*2<NN and len(list_xuhao)!=0:
                    #本次要添加的所有相关联程航班
                    l_add=[list_xuhao[0]]
                    for k in l_add:
                        list_xuhao.remove(k)
                    l_xuhao=l_xuhao+l_add
                chaifenxuhao[j]=l_xuhao
            #如果list_xuhao中还有元素，则打印‘还有元素’
            if len(list_xuhao)!=0:
                print('还有元素{}'.format(i))
                
            #将chaifenxuhao中的元素添加到pt_input[i]中
            for j in chaifenxuhao:
                pt_input[i][j]=pt[i][pt[i]['xuhao'].isin(chaifenxuhao[j])]
            
            # 检验是否都被提取出来
            # 计算所有chaifenxuhao中的元素个数和
            sum=0
            for j in chaifenxuhao:
                #print(len(chaifenxuhao[j]))
                sum+=len(chaifenxuhao[j])
            #print(sum)
            if sum!=len(pt[i]['xuhao'])/2:
                print('错误{}'.format(i))
            
            SUM=0
            for j in chaifenxuhao:
                #print(pt_input[i][j].shape[0])
                SUM+=pt_input[i][j].shape[0]
            #print(SUM)
            if SUM!=pt[i].shape[0]:
                print('错误{}'.format(i))
                
    #删除pt_input中的空字典
    for i in list(pt_input.keys()):
        for j in list(pt_input[i].keys()):
            if pt_input[i][j].shape[0]==0:
                del pt_input[i][j]
                #print('删除{}-{}'.format(i,j))
    
    return pt_input

pt_input=pt_zaichaifen(input_putong1,zaichaifen_dict)
pt_input1=pt_zaichaifen(input_putong1,zaichaifen_dict)
pt_input2=pt_zaichaifen(input_putong1,zaichaifen_dict)
#判断lc_input1和lc_input2这两个字典中的每个键对应的值是否相等
pt_zushu=0
pt_geshu=0
for i in list(pt_input1.keys()):
    for j in list(pt_input1[i].keys()):
        pt_zushu+=1
        pt_geshu+=pt_input1[i][j].shape[0]*i
        if all(pt_input1[i][j] == pt_input2[i][j]):
            pass
        else:
            print('错误{}'.format(i))
print(pt_zushu,pt_geshu)


#### 判断input1的day数据是否有问题

In [ ]:
to_delete=[]
#判断input1中的TIAN_DEP和TIAN_ARR是否相同，是否有空值
for i in input1.index.tolist():
    if input1['TIAN_DEP'][i]!=input1['TIAN_ARR'][i]:
        #如果不同则判断TIAN_ARR的所有元素减1是否等于TIAN_DEP
        if (np.array(input1['TIAN_ARR'][i])-1==np.array(input1['TIAN_DEP'][i])).all():
            pass
        else:
            #得到i对应的xuhao列的值
            xuhao=input1.loc[i,'xuhao'].tolist()
            to_delete.append(xuhao)
            print('错误{}'.format(i))
    if input1['TIAN_DEP'][i]==[]:
        #得到i对应的xuhao列的值
        xuhao=input1.loc[i,'xuhao'].tolist()
        to_delete.append(xuhao)
        print('错误'.format(i))
    if input1['TIAN_ARR'][i]==[]:
        #得到i对应的xuhao列的值
        xuhao=input1.loc[i,'xuhao'].tolist()
        to_delete.append(xuhao)
        print('错误'.format(i))
to_delete=list(set(to_delete))
print(to_delete)


#### 得到最终拆分组的字典双索引【合并得到822组按照重要度顺序排列好的dataframe，作为input】

In [ ]:
#打印出lc_input的所有键值对
lc_index={}
key=0
lcl=list(lc_input.keys())
lcl.sort(reverse=True)
for i in lcl:
    lclc=list(lc_input[i].keys())
    lclc.sort(reverse=True)
    for j in lclc:
        lc_index[key]=[i,j]
        key+=1
print(len(lc_index),lc_index)

#打印出pt_input的所有键值对
pt_index={}
key=0
lcl=list(pt_input.keys())
lcl.sort(reverse=True)
for i in lcl:
    lclc=list(pt_input[i].keys())
    lclc.sort(reverse=True)
    for j in lclc:
        pt_index[key]=[i,j]
        key+=1
print(len(pt_index),pt_index)

#得到最终的822组input
def final_input(lc_input,pt_input):
    input={}
    zuhao = 0
    for i in lc_index:
        input[zuhao] = lc_input[lc_index[i][0]][lc_index[i][1]]
        zuhao+=1
    for i in pt_index:
        input[zuhao] = pt_input[pt_index[i][0]][pt_index[i][1]]
        zuhao+=1
    print(zuhao)

    return input
input=final_input(lc_input,pt_input)


#### 得到input中30组数据每组的航班个数---流量值

In [ ]:
dict_zuhao_liuliangzhi={}
sum=0
for i in input:
    #得到input[i]的day_len列的第一个值
    number_daylen=input[i]['day_len'].tolist()[0]
    dict_zuhao_liuliangzhi[i]=input[i].shape[0]*number_daylen
    sum+=input[i].shape[0]*number_daylen
print(dict_zuhao_liuliangzhi)
print(sum)


#### 数据预处理eval：将input中的字符串列表转换为列表

In [ ]:
#将input中的字符串列表转换为列表
for ii in input:
    #print(ii)
    for i in input[ii].index.tolist():
        for j in ['t1dep','t1arr','t2dep','t2arr','Tw_it','Tx_itds','x_TIAN']:
            input[ii][j][i]=eval(input[ii][j][i])

    #a添加Tw_it_jia1和Tx_itds_jia1列,其中Tw_it_jia1为Tw_it列值加上其最后一个值加1,另外一个一样
    for i in input[ii].index.tolist():
        input[ii].loc[i,'Tw_it_jia1'] = str(input[ii].loc[i,'Tw_it']+[input[ii].loc[i,'Tw_it'][-1] + 1])
        input[ii].loc[i,'Tx_itds_jia1'] = str(input[ii].loc[i,'Tx_itds']+[input[ii].loc[i,'Tx_itds'][-1] + 1])
        #添加航班计划的相关机场，dep/arr之间不相同
        if input[ii].loc[i,'dep0/arr1']==0:
            input[ii].loc[i,'airport']=input[ii].loc[i,'depaddr']+str('-D')
        else:
            input[ii].loc[i,'airport']=input[ii].loc[i,'arraddr']+str('-A')
        #添加可能的15min运行时间片
        Tmin=int(min(input[ii].loc[i,'Tx_itds'])/3)
        Tmax=int(max(input[ii].loc[i,'Tx_itds'])/3)
        input[ii].loc[i,'Tx_iTds']=str(list(range(Tmin, Tmax+1)))

    for i in input[ii].index.tolist():
        for j in ['Tw_it_jia1','Tx_itds_jia1','Tx_iTds']:
            input[ii][j][i]=eval(input[ii][j][i])


#### 优先把初始容量处理了，在建立class时直接导入

In [ ]:
qingjing=[0,1,2]
airport51=A
airport51_dep=[i + '-D' for i in airport51]
airport51_arr=[i + '-A' for i in airport51]
airport51_dep_arr=airport51_dep+airport51_arr#索引A机场
#print(airport51_dep_arr)

#S_DEPARR是将S[0][i]于S[1][i]合并得到所有机场在所有情景下的所有时间片的容量
S_DEPARR = {}
for i in qingjing:
    #依据S[0][i]、S[1][i]列名合并得到
    S_DEPARR[i] = pd.concat([S[0][i],S[1][i]],axis=0).reset_index(drop=True)
#print(S_DEPARR[0])

#设置机场的容量索引，这也是后续需要频繁更新的数据，总共4层索引，天数、机场、情景、时间

sum=0
capacity_initial={}
for i in range(150,151):#只有一天，下面拓展到7天【初始容量】
    capacity_initial[i]={}
    for j in airport51_dep_arr:
        capacity_initial[i][j]={}
        for k in qingjing:
            capacity_initial[i][j][k]={}
            for l in range(0,96):
                #得到机场j、情景k、l列的值
                C=S_DEPARR[k][S_DEPARR[k]['airport']==j].loc[:,str(l)].values[0]
                capacity_initial[i][j][k][l]=C
                sum=sum+C
print(sum*3)

import copy
#复制key值为0的dict_day_qingjing_airport_15minindex_capacity，得到key值为1到209的dict_day_qingjing_airport_15minindex_capacity
for i in range(151,153):#拓展到2天
    capacity_initial[i]=copy.deepcopy(capacity_initial[150])

#判断每个key对应的value是否相同
for i in range(150,153):#判断容量数据在拓展后是否正确
    if capacity_initial[i]==capacity_initial[150]:
        continue
    else:
        print('错误{}'.format(i))
print(capacity_initial)


#### 创建空字典作为类变量，统计qiujie已使用的容量\diedai的剩余容量

In [ ]:
#d是一个4索引字典，我需要创建一个空的4索引字典，和d一样的结构，但是值为0
def create_empty_dict(capacity_initial):
    dict_empty_qiujie={}
    dict_empty_youhua={}
    for i in capacity_initial:
        dict_empty_qiujie[i]={}
        dict_empty_youhua[i]={}
        for j in capacity_initial[i]:
            dict_empty_qiujie[i][j]={}
            dict_empty_youhua[i][j]={}
            for k in capacity_initial[i][j]:
                dict_empty_qiujie[i][j][k]={}
                dict_empty_youhua[i][j][k]={}
                for l in capacity_initial[i][j][k]:
                    dict_empty_qiujie[i][j][k][l]=0
                    dict_empty_youhua[i][j][k][l]=0
    return dict_empty_qiujie,dict_empty_youhua
dict_empty_qiujie,dict_empty_youhua=create_empty_dict(capacity_initial)


#### 按照机场层面的重要度，建立所有航班xuhao-dep0/arr1的重要度字典，model2直接导入

In [ ]:
#将input中所有键对应的value合并为一个dataframe
input_all=pd.concat([input[i] for i in input],axis=0).reset_index(drop=True)
dict_xuhao_airport={}
for i in input_all.index.tolist():
    if input_all.loc[i,'dep0/arr1']==0:
        dict_xuhao_airport[input_all.loc[i,'xuhao-dep0/arr1']]=input_all.loc[i,'depaddr']
    else:
        dict_xuhao_airport[input_all.loc[i,'xuhao-dep0/arr1']]=input_all.loc[i,'arraddr']
# #print(list(dict_xuhao_airport.items())[0:2])
# delay_importance=pd.read_csv('../../51个机场的延误重要度/delay_importance_gui1hua.csv')
# dict_xuhao_delay_importance={}
# for i in dict_xuhao_airport:
#     air=dict_xuhao_airport[i]
#     #如果air在delay_importance的airport列中
#     if air in delay_importance['airport'].tolist():
#         dict_xuhao_delay_importance[i]=delay_importance[delay_importance['airport']==air]['delay'].values[0]
#     else:
#         dict_xuhao_delay_importance[i]=0.8
# list(dict_xuhao_delay_importance.items())


#### 将所有分组再次合并，进行测试【测试gap时需要整体计算】【一定要重置index，否则很可能报错】

In [ ]:
#将input的所有数据合并为一个dataframe
sum=0
input_total=pd.DataFrame()
for i in range(0,97):#这个组数需要更新
    sum+=input[i].shape[0]
    input_total=pd.concat([input_total,input[i]],axis=0)
input_total=input_total.reset_index(drop=True)

#判断是否有相同的索引值
print(input_total.index.is_unique)
print(input_total.shape)


In [ ]:
#input_total中airport列的值在airport51_dep_arr中行的day_len列之和
sum=0
for i in input_total.index.tolist():
    if input_total.loc[i,'airport'] in airport51_dep_arr:
        sum+=input_total.loc[i,'day_len']
print(sum*3)


#### 检验input_total中是否有错误数据，如：第153天、288时间片，96时间片之类的

In [ ]:
list_xiugai=[]
#判断input_total中的x_TIAN列的列表里是否有210
for i in input_total.index.tolist():
    if 153 in input_total.loc[i,'x_TIAN']:
        xuhao_wenti=input_total.loc[i,'xuhao']
        #print('错误{}'.format(xuhao_wenti))
        list_xiugai.append(xuhao_wenti)
print(len(list_xiugai))
print(list_xiugai)

max_relation=[]
for i in list_xiugai:
    if i in list(dict_max_relation_xuhao.keys()):
        add=dict_max_relation_xuhao[i]
        max_relation+=add
    else:
        max_relation.append(i)
max_relation=list(set(max_relation))
print(len(max_relation))
print(max_relation)


#### xuhao-dep0/arr1与airport的对应字典

In [ ]:
#建立一个序号和机场之间的字典
dict_xuhao_jichang={}
for i in input_total.index.tolist():
    dict_xuhao_jichang[input_total.loc[i,'xuhao-dep0/arr1']]=input_total.loc[i,'airport']
print(len(dict_xuhao_jichang))
print(list(dict_xuhao_jichang.items())[0:2])


#### 优先计算好所有xuhao-dep0/arr1的可能出现的机场、天数、15min时间片【大幅降低class.presolve的时间】

In [ ]:
#建立3索引字典，用机场、天数和15min时间片来索引航班序号，用机场、天数和15min时间片来索引航班序号
dict_airport_day_15min_xuhaodep0arr1_total={}
for i in airport51_dep_arr:
    dict_airport_day_15min_xuhaodep0arr1_total[i]={}
    #得到a的airport列为i的行
    a1 = input_total[input_total['airport'] == i]
    #如果a1为空dataframe
    if a1.empty:
        for j in range(150,153):
            dict_airport_day_15min_xuhaodep0arr1_total[i][j]={}
            for n in range(0,96):
                dict_airport_day_15min_xuhaodep0arr1_total[i][j][n]=[]
        #进入下一个i循环
        continue
    else:
        for j in range(150,153):
            dict_airport_day_15min_xuhaodep0arr1_total[i][j]={}
            #a2为a1中x_TIAN列值包含j元素的行
            a2 = a1[a1['x_TIAN'].apply(lambda x: j in x)]
            if a2.empty:
                for n in range(0,96):
                    dict_airport_day_15min_xuhaodep0arr1_total[i][j][n]=[]
                continue
            else:
                for n in range(0,96):
                    dict_airport_day_15min_xuhaodep0arr1_total[i][j][n]=[]
                    a3 = a2[a2['Tx_iTds'].apply(lambda x: n in x)]
                    if a3.empty:
                        continue
                    else:
                        for k in a3.index.tolist():
                            dict_airport_day_15min_xuhaodep0arr1_total[i][j][n].append(a3['xuhao-dep0/arr1'][k])

# #容量约束中的转化字典，替代函数【这是不可行的，gurobi约束中不可以使用决策变量作为字典的索引】
print(dict_airport_day_15min_xuhaodep0arr1_total['ZBAA-D'][150][0])#机场、天数、15min时间片


### 所需的数据处理

In [ ]:
# print(len(dict_xuhao_delay_importance))

# li=[]
# #将dict_xuhao_delay_importance中重要度为0.8的元素放在li列表中
# for i in dict_xuhao_delay_importance:
#     if dict_xuhao_delay_importance[i]==0.8:
#         li.append(i)
# print(len(li))

chushi_QINGKUANG=pd.DataFrame(columns=['q','time'])#每次迭代都变，记录迭代情况

In [ ]:
#建立一个序号xuhao-dep0arr1和运行天数之间的字典                         ###
dict_xuhao_tianshu={}
for i in input_total.index.tolist():
    dict_xuhao_tianshu[input_total.loc[i,'xuhao-dep0/arr1']]=input_total.loc[i,'day_len']
print(len(dict_xuhao_tianshu))
print(list(dict_xuhao_tianshu.items())[0:2])

In [ ]:
#建立一个序号xuhao-dep0arr1和运行机场之间的字典                         ###
dict_xuhaodep0arr1_airport={}
for i in input_total.index.tolist():
    dict_xuhaodep0arr1_airport[input_total.loc[i,'xuhao-dep0/arr1']]=input_total.loc[i,'airport']
print(len(dict_xuhaodep0arr1_airport))
print(list(dict_xuhaodep0arr1_airport.items())[0:2])

In [ ]:
timewindow_geshu=95
#设置时间窗的suoyin和5min时间片的对应关系                              ###
dict_time_window={}
#设置95个时间窗的索引
for i in range(0,timewindow_geshu):
    dict_time_window[i]=list(range(3*i,3*i+6))
print(dict_time_window[timewindow_geshu-1])

In [ ]:
#建立xuhao-dep0/arr1和Tfp之间的字典关系                               ###
dict_xuhao_Tfp={}
for i in input_total.index.tolist():
    dict_xuhao_Tfp[input_total.loc[i,'xuhao-dep0/arr1']]=input_total.loc[i,'Tfp']
# print(len(dict_xuhao_Tfp))
# print(dict_xuhao_Tfp['0-0'])

In [ ]:
dict_time_window_xuhaodep0arr1={}
for i in range(0,timewindow_geshu):
    ll=[]
    for j in dict_xuhao_Tfp:
        if dict_xuhao_Tfp[j] in dict_time_window[i]:
            ll.append(float(j[0:-2]))
            ll=list(set(ll))
    dict_time_window_xuhaodep0arr1[i]=ll
print((dict_time_window_xuhaodep0arr1[0]))
print(len(dict_time_window_xuhaodep0arr1[0]))

In [ ]:
#依据dict_max_relation_xuhao进一步拓展各个时间窗内的相关航班                            ###
dict_time_window_xuhao={}
for i in range(0,timewindow_geshu):
    for j in dict_time_window_xuhaodep0arr1[i]:#对于每个时间窗内的航班xuhao进行循环
        add=[]
        #如果j是dict_max_relation_xuhao的键
        if j in dict_max_relation_xuhao.keys():
            for k in dict_max_relation_xuhao[j]:
                add.append(float(k))
        xx=dict_time_window_xuhaodep0arr1[i]+add
        xx=list(set(xx))
        dict_time_window_xuhao[i]=xx
print(len(dict_time_window_xuhao[0]))

In [ ]:
#每个时间窗内的航班xuhao-dep0/arr1的数量【它是xuhao数的2倍】                            ###
dict_sum={}
dict_slotsum={}
for i in range(0,timewindow_geshu):#对所有时间窗进行循环，得到每个时间窗内的航班xuhao的数量，以及时间窗内的航班slot数量
    dict_sum[i]=len(dict_time_window_xuhao[i])
    dict_slotsum[i] = 0#时间窗内的航班slot数量
    for j in dict_time_window_xuhao[i]:
        dict_slotsum[i]+=input_total[input_total['xuhao']==j]['day_len'].values[0]*2#每个xuhao有dep/arr这2部分组成，所以要乘以2
print('时间窗内的xuhao数量字典{}'.format(dict_sum))
print('时间窗内的slot数量字典{}'.format(dict_slotsum))

In [ ]:
xuhao_number=200
#设置xuhao初始选择比例                                ###
dict_time_window_bili={}
for i in dict_slotsum:
    dict_time_window_bili[i]=xuhao_number/dict_sum[i]#限制每个优化组的时刻选取数量
    if dict_time_window_bili[i]>1:
        dict_time_window_bili[i]=1
print('时间窗内的初始xuhao选择bili{}'.format(dict_time_window_bili))

### 迭代算法开始时需要的数据处理

In [ ]:
#制作航班序号和目标值的对应关系                                      ###
dict_xuhaodep0arr1_obj={}
dict_xuhaodep0arr1_objzl={}#战略偏移
dict_xuhaodep0arr1_objzs={}#战术延误
sum=0# 统计延误
sum1=0# 统计提前【这部分没有算在延误里，归为0了】
sum2=0# 统计战略目标值

sumsum=0
sum11=0
sum22=0

for iii in range(0,97):
    #制作航班序号和目标值的对应关系                                      ###
    dict_xuhaodep0arr1_zlobj={}
    #读入单个的sol文件
    with open('../../output/two-stage/1/chushijie/{}.sol'.format(iii), "r") as f:
        f.seek(0)#重新定义文件指针【每次只能读取一次】
        for line in f:
            if line[0]=='A':
                dict_xuhaodep0arr1_zlobj[line.split(' ')[0].split('[')[1][:-1]]=float(line.split(' ')[1])
    keys=list(dict_xuhaodep0arr1_zlobj.keys())
    #print(len(keys))
    keys1=[]
    dict_xuhaodep0arr1_zlobj1={}
    for l in keys:
        keys1.append(l.split(',')[0])
    keys1=list(set(keys1))
    #print(len(keys1))
    for l in keys1:
        ll=[]
        for k in keys:
            if k.split(',')[0]==l:
                ll.append(dict_xuhaodep0arr1_zlobj[k])
                llsum=0
                for j in ll:
                    llsum+=abs(j)
        dict_xuhaodep0arr1_zlobj1[l]=llsum
    # print(dict_xuhaodep0arr1_zlobj1)
    summmmm=0
    for i in dict_xuhaodep0arr1_zlobj1:
        summmmm+=abs(dict_xuhaodep0arr1_zlobj1[i])
    summmmmm=0
    for i in dict_xuhaodep0arr1_zlobj:
        summmmmm+=abs(dict_xuhaodep0arr1_zlobj[i])    
    dict_xuhaodep0arr1_zlobj=dict_xuhaodep0arr1_zlobj1
    
    # print(len(dict_xuhaodep0arr1_zlobj))
    # #求出所有value的和
    for i in dict_xuhaodep0arr1_zlobj:
        sum2+=abs(dict_xuhaodep0arr1_zlobj[i])
        if i in li:
            sum22+=abs(dict_xuhaodep0arr1_zlobj[i])

    #制作航班序号和目标值的对应关系                                      ###
    dict_xuhaodep0arr1_zsobj={}
    dict_jianjie={}
    #读入单个的sol文件
    with open('../../output/two-stage/1/chushijie/{}.sol'.format(iii), "r") as f:
        f.seek(0)#重新定义文件指针【每次只能读取一次】
        zs_obj=0
        for line in f:
            if line[0]=='Y':
                dict_jianjie[tuple([line.split(' ')[0].split(',')[0][6:],line.split(' ')[0].split(',')[1],line.split(' ')[0].split(',')[2][:-1]])]=float(line.split(' ')[1])
    # print(len(dict_jianjie))
    # print(list(dict_jianjie.items())[0:1][0])

    xuhao_input=input[iii]['xuhao-dep0/arr1'].tolist()
    for xuhaoi in xuhao_input:#后续执行循环
        zs_obj=0
        for i in dict_jianjie:
            if i[0]==xuhaoi:
                if dict_jianjie[i]>0:
                    zs_obj+=dict_jianjie[i]*qingjing_weight[int(i[2])]
        dict_xuhaodep0arr1_zsobj[xuhaoi]=zs_obj
    #print(dict_xuhaodep0arr1_zsobj)

    for i in dict_jianjie:
        if dict_jianjie[i]>0:
            sum+=dict_jianjie[i]*qingjing_weight[int(i[2])]
            if i[0] in li:
                sumsum+=dict_jianjie[i]*qingjing_weight[int(i[2])]
        else:
            sum1+=dict_jianjie[i]*qingjing_weight[int(i[2])]
            if i[0] in li:
                sum11+=dict_jianjie[i]*qingjing_weight[int(i[2])]

    for i in dict_xuhaodep0arr1_zlobj:
        dict_xuhaodep0arr1_obj[i]=abs(dict_xuhaodep0arr1_zlobj[i])+dict_xuhaodep0arr1_zsobj[i]
        #添加xuhaodep0arr1_objzl和xuhaodep0arr1_objzs字典
        dict_xuhaodep0arr1_objzl[i]=dict_xuhaodep0arr1_zlobj[i]
        dict_xuhaodep0arr1_objzs[i]=dict_xuhaodep0arr1_zsobj[i]
print('目标函数中的战术延误值为：{}'.format(sum))
print('未统计进入目标函数的战术提前值为：{}'.format(sum1))
print('目标函数中的战略偏移值为：{}'.format(sum2))
print('--------------------------------------------')
print('li的目标函数中的战术延误值为:{}'.format(sumsum))
print('li的未统计进入目标函数的战术提前值为:{}'.format(sum11))
print('li的目标函数中的战略偏移为:{}'.format(sum22))

dict_xuhao_weight={}#这就是xuhao的权重
oo=0
for i in list(set(input1['xuhao'].tolist())):
    o=0
    #得到input1中的xuhao对应的xuhao-dep0/arr1列值的列表
    list_xuhao_dep0arr1=input1[input1['xuhao']==i]['xuhao-dep0/arr1'].tolist()
    for j in list_xuhao_dep0arr1:
        o+=dict_xuhaodep0arr1_obj[j]
        oo+=dict_xuhaodep0arr1_obj[j]
    dict_xuhao_weight[i]=o

#计算时间窗内的目标函数值
time_window_weight={}
for i in range(0,timewindow_geshu):
    sum=0
    for j in dict_time_window_xuhao[i]:
        sum+=dict_xuhao_weight[j]
    time_window_weight[i]=sum
print('所有时间窗的目标函数值：{}'.format(time_window_weight))#所有时间窗的目标函数值

sum=0
for i in dict_xuhao_weight:
    sum+=dict_xuhao_weight[i]
print('目标函数值：{}'.format(sum))#所有xuhao的目标函数值


In [ ]:
#制作航班序号和V_iTds的对应关系                                   ###
dict_xuhaodep0arr1_V={}
num=0
for iii in range(0,97):
    #读入单个的sol文件
    with open('../../output/two-stage/1/chushijie/{}.sol'.format(iii), "r") as f:
        f.seek(0)#重新定义文件指针【每次只能读取一次】
        for line in f:
            if line[0]=='V' and line.split(' ')[1][0]=='1':
                xh=line.split(' ')[0].split(',')[0][7:]
                V=[xh,float(line.split(' ')[0].split(',')[1]),float(line.split(' ')[0].split(',')[2]),
                   float(line.split(' ')[0].split(',')[3][0]),dict_xuhaodep0arr1_airport[xh]]
                dict_xuhaodep0arr1_V[num]=V
                num+=1

# 创建一个空列表来存储要删除的键【不在51个机场之中】
keys_to_delete = []
#将机场不在airport51_dep_arr中的item删除
for i in dict_xuhaodep0arr1_V:
    if dict_xuhaodep0arr1_V[i][4] not in airport51_dep_arr:
        keys_to_delete.append(i)
# 遍历要删除的键的列表
for key in keys_to_delete:
    # 删除键
    dict_xuhaodep0arr1_V.pop(key)

#初始解已使用的容量                         ###
dict_usedd=copy.deepcopy(dict_empty_qiujie)
for i in dict_xuhaodep0arr1_V:#字典的key值
    dict_V_xuhaodep0arr1=dict_xuhaodep0arr1_V[i][0]#xuhao-dep0/arr1
    dict_V_time15=dict_xuhaodep0arr1_V[i][1]#15min时间片
    dict_V_day=dict_xuhaodep0arr1_V[i][2]#天数
    dict_V_qingjing=dict_xuhaodep0arr1_V[i][3]#情景
    dict_V_jichang=dict_xuhaodep0arr1_V[i][4]#机场
    dict_usedd[dict_V_day][dict_V_jichang][dict_V_qingjing][dict_V_time15]+=1

dict_capacity_initial=copy.deepcopy(capacity_initial)
sum1=0
for i in dict_capacity_initial:
    for j in dict_capacity_initial[i]:
        for k in dict_capacity_initial[i][j]:
            for l in dict_capacity_initial[i][j][k]:
                sum1+=dict_capacity_initial[i][j][k][l]
print('初始容量总数{}'.format(sum1))#求初始解的总容量

dict_capacity_remain_diedai=copy.deepcopy(capacity_initial)#【类变量】

#更新初始容量，减去已经使用的容量
sum3=0
for i in dict_capacity_remain_diedai:
    for j in dict_capacity_remain_diedai[i]:
        for k in dict_capacity_remain_diedai[i][j]:
            for l in dict_capacity_remain_diedai[i][j][k]:
                dict_capacity_remain_diedai[i][j][k][l]-=dict_usedd[i][j][k][l]
                if dict_capacity_remain_diedai[i][j][k][l]<0:
                    print('错误{},{},{},{},少{}'.format(i,j,k,l,dict_capacity_remain_diedai[i][j][k][l]))
                else:
                    sum3+=dict_capacity_remain_diedai[i][j][k][l]

dict_xuhao_obj=copy.deepcopy(dict_xuhao_weight)

diedai_QINGKUANG=pd.DataFrame(columns=['q','time_window','obj','obj_zl','obj_zs','zuiyou','youjie','wujie','TW_bili','TW_bili_biangeng',
                                           'TW_weight','TW_weight_biangeng','shifouyouhua','time'])#每次迭代都变，记录迭代情况


### 转化为class

#### 更新已用容量【可在迭代中断、重跑时进行】

In [ ]:
VV=[]#得到.sol中V_iTds为1的值，即航班i在d天情景s下的战术运行时刻T15min
dict_VV={}#提取V中元素对应的航班序号、时间片、天数、情景和机场信息到字典dict_V

sol_files_directory = '../../output/two-stage/1/chushijie/'  # 存放.sol文件的目录
#sol_files_directory = '../../output/two-stage/1/zhijiejie/'  # 存放.sol文件的目录
# 循环处理所有.sol文件
for filename in os.listdir(sol_files_directory):
    # if filename.endswith(".sol"):
    file_path = os.path.join(sol_files_directory, filename)
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # #得到.sol中V_iTds为1的值，即航班i在d天情景s下的战术运行时刻T15min
        for line in lines:
            if line[0] == 'V' and line[-2] == '1':
                VV.append(line)

#提取V中元素对应的航班序号、时间片、天数、情景和机场信息到字典dict_V
dict_VV={}
for i in range(0,len(VV)):
    air=dict_xuhao_jichang[VV[i].split(',')[0][7:]]
    #如果air不在airport51_dep_arr中，则跳过
    if air not in airport51_dep_arr:
        continue
    else:
        dict_VV[i]=[VV[i].split(',')[0][7:],float(VV[i].split(',')[1]),
                        float(VV[i].split(',')[2]),float(VV[i].split(',')[3][0]),air]
print('dict_VV的长度为{}'.format(len(dict_VV)))

used=copy.deepcopy(dict_empty_qiujie)
for i in dict_VV:
    dict_V_xuhaodep0arr1=dict_VV[i][0]#xuhao-dep0/arr1
    dict_V_time15=dict_VV[i][1]#15min时间片
    dict_V_day=dict_VV[i][2]#天数
    dict_V_qingjing=dict_VV[i][3]#情景
    dict_V_jichang=dict_VV[i][4]#机场
    used[dict_V_day][dict_V_jichang][dict_V_qingjing][dict_V_time15]+=1

sum1=0
for i in used:
    for j in used[i]:
        for k in used[i][j]:
            for l in used[i][j][k]:
                sum1+=used[i][j][k][l]
print(sum1)


#### 中断

In [ ]:
# zhongduan_time_window_weight=pd.read_csv('../../output/two-stage/time_window_weight/yuan96.csv')
# zhongduan_dict_time_window_bili=pd.read_csv('../../output/two-stage/time_window_bili/yuan96.csv')
# zhongduan_dict_xuhao_weight=pd.read_csv('../../output/two-stage/xuhao_weight/yuan96.csv')
# zhongduan_dict_xuhao_obj=pd.read_csv('../../output/two-stage/xuhao_obj/yuan96.csv')
# diedai_QINGKUANG=pd.read_csv('../../output/two-stage/diedaixinxi/diedai_qingkuang96.csv')
# zhongduan_dict_xuhaodep0arr1_objzl=pd.read_csv('../../output/two-stage/diedaimubiaozl/yuan96.csv')
# zhongduan_dict_xuhaodep0arr1_objzs=pd.read_csv('../../output/two-stage/diedaimubiaozs/yuan96.csv')
# time_window_weight={}
# for i in range(len(zhongduan_time_window_weight)):
#     time_window_weight[zhongduan_time_window_weight.iloc[i,0]]=zhongduan_time_window_weight.iloc[i,1]
# dict_time_window_bili={}
# for i in range(len(zhongduan_dict_time_window_bili)):
#     dict_time_window_bili[zhongduan_dict_time_window_bili.iloc[i,0]]=zhongduan_dict_time_window_bili.iloc[i,1]
# dict_xuhao_weight={}
# for i in range(len(zhongduan_dict_xuhao_weight)):
#     dict_xuhao_weight[zhongduan_dict_xuhao_weight.iloc[i,0]]=zhongduan_dict_xuhao_weight.iloc[i,1]
# dict_xuhao_obj={}
# for i in range(len(zhongduan_dict_xuhao_obj)):
#     dict_xuhao_obj[zhongduan_dict_xuhao_obj.iloc[i,0]]=zhongduan_dict_xuhao_obj.iloc[i,1]
# zhongduan_dict_capacity_remain_diedai=copy.deepcopy(dict_capacity_remain_diedai)
# for i in [150,151,152]:
#     for j in [0,1,2]:
#         zhongduan_C=pd.read_csv('../../output/two-stage/capacity_remain_diedai_benci/c{}_{}.csv'.format(i,j))
#         zhongduan_C.index=airport51_dep_arr
#         for ii in airport51_dep_arr:
#             for jj in range(0,96):
#                 zhongduan_dict_capacity_remain_diedai[i][ii][j][jj]=zhongduan_C.loc[ii,str(jj)]
# dict_capacity_remain_diedai=zhongduan_dict_capacity_remain_diedai
# zhongduan_dict_xuhaodep0arr1_V={}
# num=0
# nuu=0
# a='../../output/two-stage/xuhaodep0arr1_V_suoyou/yuan.json'
# import json
# with open(a,'r') as f:
#     dict_xuhaodep0arr1_V = json.load(f)
# for i,j in dict_xuhaodep0arr1_V.items():
#     if '-' in i:
#         num+=1
#         continue
#     else:
#         zhongduan_dict_xuhaodep0arr1_V[int(i)]=j
#     nuu+=1
# dict_xuhaodep0arr1_V=zhongduan_dict_xuhaodep0arr1_V
# print(num,nuu)
# dict_xuhaodep0arr1_objzl={}
# for i in range(len(zhongduan_dict_xuhaodep0arr1_objzl)):
#     dict_xuhaodep0arr1_objzl[zhongduan_dict_xuhaodep0arr1_objzl.iloc[i,0]]=zhongduan_dict_xuhaodep0arr1_objzl.iloc[i,1]
# dict_xuhaodep0arr1_objzs={}
# for i in range(len(zhongduan_dict_xuhaodep0arr1_objzs)):
#    dict_xuhaodep0arr1_objzs[zhongduan_dict_xuhaodep0arr1_objzs.iloc[i,0]]=zhongduan_dict_xuhaodep0arr1_objzs.iloc[i,1]

#### class

In [ ]:
class MODEL:
    #求解阶段
    #定义类变量，这些会随着每次迭代而变化【均对引用添加deepcopy，确保被引用的外部嵌套字典不变化】
    #capacity_used_qiujie=copy.deepcopy(dict_empty_qiujie)#每次迭代都变，用来记录总体的容量使用情况【这里是字典套字典的特殊情况，浅拷贝还是会变化】
    capacity_used_qiujie=copy.deepcopy(used)#容量调整时暂时使用这个【这样可以从断开处继续计算】
    # capacity_remain_youhua=copy.deepcopy(dict_empty_youhua)#每次迭代都变，用来记录总体的容量剩余情况
    #设置每次开始迭代时的目标函数值
    objective=0

    # #优化阶段    #这里需要跑算法2
    time_window_weight0=time_window_weight#每次迭代都变，用来记录总体的时间窗权重情况
    time_window_bili=dict_time_window_bili#每次迭代都变，用来记录总体的时间窗比例情况
    dict_xuhao_weight0=dict_xuhao_weight#每次迭代都变，用来记录航班xuhao的权重情况
    dict_xuhao_obj0=dict_xuhao_obj#每次迭代都变，用来记录航班xuhao的目标函数值【每次要比较目标函数，战略战术都要正的】
    dict_capacity_remain_diedai_benci=dict_capacity_remain_diedai#每次迭代都变，得到迭代解后记录总体的容量剩余情况【得到所有解之后】
    dict_xuhaodep0arr1_V_suoyou=dict_xuhaodep0arr1_V#每次迭代都变，更新所有航班的容量使用情况
    diedai_qingkuang=diedai_QINGKUANG#这就是那个表格，记录每次迭代的情况
    chushi_qingkuang=chushi_QINGKUANG#这就是那个表格，记录每次迭代的情况
    dict_xuhaodep0arr1_objzl0=dict_xuhaodep0arr1_objzl#用于记录每次迭代的航班xuhao-dep0arr1的战略目标函数值【有负的】
    dict_xuhaodep0arr1_objzs0=dict_xuhaodep0arr1_objzs#用于记录每次迭代的航班xuhao-dep0arr1的战术目标函数值【都是正的】

    #initial里的数据都是不随class的循环迭代而变化的数据【输入参数的顺序很重要，之前出现过容量赋值一直为0的情况，就是参数输入顺序问题】
    def __init__(self,III,input,A,capacity_initial,qingjing,qingjing_weight,dict_xuhao_jichang,dict_xuhao_delay_importance,
                 dict_airport_day_15min_xuhaodep0arr1_total,dict_zuhao_liuliangzhi,dict_sum,dict_time_window_xuhao,dict_max_relation_xuhao,
                 input_total,airport51_dep_arr,dict_xuhaodep0arr1_airport,dict_xuhao_tianshu):
        self.m=gp.Model()
        self.III=III
        self.input=input
        self.A=A#考虑容量限制的机场集合
        self.capacity_initial=copy.deepcopy(capacity_initial)#每次调用模型时都是初始容量
        self.qingjing=qingjing#情景集合
        self.qingjing_weight=qingjing_weight#情景权重
        self.dict_xuhao_jichang=dict_xuhao_jichang#航班与机场的对应关系
        self.dict_xuhao_delay_importance=dict_xuhao_delay_importance#航班的延误重要度
        self.dict_airport_day_15min_xuhaodep0arr1_total=dict_airport_day_15min_xuhaodep0arr1_total#机场、天数、15min时间片索引的航班序号
        self.dict_zuhao_liuliangzhi=dict_zuhao_liuliangzhi#input中30组，每组的航班的流量值
        self.dict_sum=dict_sum#每个时间窗的总xuhao数量
        self.dict_time_window_xuhao=dict_time_window_xuhao#时间窗的航班xuhao
        self.dict_max_relation_xuhao=dict_max_relation_xuhao#每个航班的最大关联航班序号
        self.input_total=input_total#3天的所有数据
        self.airport51_dep_arr=airport51_dep_arr#考虑容量的机场集合
        self.dict_xuhaodep0arr1_airport=dict_xuhaodep0arr1_airport#航班与机场的对应关系
        self.dict_xuhao_tianshu=dict_xuhao_tianshu#航班xuhao-dep0arr1运行的天数

    def input_qiujie(self):
        self.a=self.input[self.III].copy()
        print('第{}组数据包含{}航班流量'.format(self.III,self.dict_zuhao_liuliangzhi[self.III]))
        #重新设置索引
        self.a=self.a.reset_index(drop=True)

    def update_qiujie_benci_capacity(self):
        self.capacity_qiujie_benci=copy.deepcopy(self.capacity_initial)#每次调用模型时都是初始容量，但会被capacity_used_qiujie缩减
        
        #计算初始容量总和，这个应该是不变的
        self.sum1=0
        for i in self.capacity_qiujie_benci:
            for j in self.capacity_qiujie_benci[i]:
                for k in self.capacity_qiujie_benci[i][j]:
                    for l in self.capacity_qiujie_benci[i][j][k]:
                        self.sum1+=self.capacity_qiujie_benci[i][j][k][l]
        print('计算第{}组的初始重置容量总和{}'.format(self.III,self.sum1))
        
        #计算已用容量总和，这个应该是变化的
        self.sum2=0
        for i in MODEL.capacity_used_qiujie:
            for j in MODEL.capacity_used_qiujie[i]:
                for k in MODEL.capacity_used_qiujie[i][j]:
                    for l in MODEL.capacity_used_qiujie[i][j][k]:
                        self.sum2+=MODEL.capacity_used_qiujie[i][j][k][l]
        print('计算第{}组时已用容量总和{}'.format(self.III,self.sum2))
        
        #更新初始容量，减去已经使用的容量
        for i in self.capacity_qiujie_benci:
            for j in self.capacity_qiujie_benci[i]:
                for k in self.capacity_qiujie_benci[i][j]:
                    for l in self.capacity_qiujie_benci[i][j][k]:
                        self.capacity_qiujie_benci[i][j][k][l]-=MODEL.capacity_used_qiujie[i][j][k][l]
        
        #计算剩余容量总和，本次求解使用的容量
        self.sum3=0
        for i in self.capacity_qiujie_benci:
            for j in self.capacity_qiujie_benci[i]:
                for k in self.capacity_qiujie_benci[i][j]:
                    for l in self.capacity_qiujie_benci[i][j][k]:
                        self.sum3+=self.capacity_qiujie_benci[i][j][k][l]
        print('第{}组的本次剩余容量总和{}'.format(self.III,self.sum3))

    def input_diedai(self):
        print('第{}次迭代'.format(self.III))
        #依据dict_time_window_weight生成dict_time_window_weight的value值最大的时间窗序号
        self.time_window_weight_max=sorted(MODEL.time_window_weight0.items(),key=lambda x:x[1],reverse=True)[0][0]#【类字典】
        print('本次迭代选择的时间窗为{}号时间窗'.format(self.time_window_weight_max))
        #得到本次时间窗的时刻选择比例
        self.bili=MODEL.time_window_bili[self.time_window_weight_max]#【类字典】
        print('本次迭代选择的时间窗xuhao比例为{}'.format(self.bili))
        #计算本次要选择的xuhao个数【向下取整】
        self.xuhao_num=int(self.dict_sum[self.time_window_weight_max]*self.bili)
        print('本次迭代时间窗总共包含{}个xuhao'.format(self.dict_sum[self.time_window_weight_max]))
        print('本次迭代选择时间窗内xuhao个数为{}'.format(self.xuhao_num))
        #得到本次筛选得到的时间窗内的所有航班序号xuhao
        self.xuhao_select_time_window=self.dict_time_window_xuhao[self.time_window_weight_max]
        #得到时间窗内所有航班xuhao的权重字典
        self.xuhao_weight={}
        for i in self.xuhao_select_time_window:
            if i in list(MODEL.dict_xuhao_weight0.keys()):
                self.xuhao_weight[i]=MODEL.dict_xuhao_weight0[i]
            else:#【总有特例，有一些可能得不到，直接置0了】
                self.xuhao_weight[i]=0
        #根据xuhao_weight的值，得到目标函数值排序前xuhao_num的xuhao序号
        self.xuhao_weight_sort=sorted(self.xuhao_weight.items(),key=lambda x:x[1],reverse=True)[0:self.xuhao_num]
        self.xuhao_select=[i[0] for i in self.xuhao_weight_sort]
        
        #依据self.dict_max_relation_xuhao得到xuhao_select的所有关联xuhao【这是关联之外的关联，并不算是时间窗内部的xuhao，只是为了求解而已】
        self.ll=len(self.xuhao_select)
        self.lll=0
        while self.lll!=self.ll:
            self.lll=self.ll
            self.xuhao_max_relation=[]#用于存储联程航班
            for i in self.xuhao_select:
                if i in self.dict_max_relation_xuhao.keys():
                    self.xuhao_max_relation+=self.dict_max_relation_xuhao[i]
            #得到所有关联xuhao
            self.xuhao_select=self.xuhao_select+self.xuhao_max_relation
            self.xuhao_select=list(set(self.xuhao_select))
            self.ll=len(self.xuhao_select)
        print('本次迭代最终选择的最大关联xuhao个数为{}'.format(len(self.xuhao_select)))
        #得到self.a作为本次迭代优化的输入【依据self.xuhao_select进行筛选】
        self.input_youhua=self.input_total[self.input_total['xuhao'].isin(self.xuhao_select)]
        self.input_youhua=self.input_youhua.reset_index(drop=True)
        self.a=self.input_youhua.copy()
        print('本次迭代最终输入的xuhao-dep0arr1的个数为{}'.format(len(self.a)))
        sum=0
        for i in self.xuhao_select:
            sum+=self.dict_xuhao_obj0[i]
        print('本次迭代最终输入的xuhao-dep0arr1的目标函数值为{}'.format(sum))
        print('------------------------------------------------------------------------------')
        print('------------------------------------------------------------------------------')

    def update_diedai_benci_capacity(self):
        #将dict_xuhaodep0arr1_airport在airport51_dep_arr中的保留下来
        self.benci_diedai_xuhaodep0arr1=self.a['xuhao-dep0/arr1'].tolist()
        print('本次迭代输入的xuhao-dep0arr1的个数为{}'.format(len(self.benci_diedai_xuhaodep0arr1)))
        self.huohua_rongliang_xuhaodep0arr1=[]
        for i in self.benci_diedai_xuhaodep0arr1:
            if self.dict_xuhaodep0arr1_airport[i] in self.airport51_dep_arr:
                self.huohua_rongliang_xuhaodep0arr1.append(i)
        print('本次迭代需要计算容量的xuhao-dep0arr1的个数为{}'.format(len(self.huohua_rongliang_xuhaodep0arr1)))
        self.sum=0
        #统计本次迭代需要计算容量的xuhao-dep0arr1的总流量
        for i in self.huohua_rongliang_xuhaodep0arr1:
            self.sum+=self.a[self.a['xuhao-dep0/arr1']==i]['day_len'].values[0]
        print('本次迭代需要计算容量的slot的总流量为{}'.format(self.sum))
        
        #得到huohua_rongliang_xuhaodep0arr1的所有key
        self.key_huohua=[]
        for i in MODEL.dict_xuhaodep0arr1_V_suoyou:
            if MODEL.dict_xuhaodep0arr1_V_suoyou[i][0] in self.huohua_rongliang_xuhaodep0arr1:
                self.key_huohua.append(i)
        #key_huohua中的值为key，将dict_xuhaodep0arr1_V中的对应的value添加到dict_xuhaodep0arr1_V_diedai_benci中
        self.dict_xuhaodep0arr1_V_diedai_benci={}
        for i in self.key_huohua:
            self.dict_xuhaodep0arr1_V_diedai_benci[i]=MODEL.dict_xuhaodep0arr1_V_suoyou[i]
        print('本次迭代实际活化的容量个数{}'.format(len(self.dict_xuhaodep0arr1_V_diedai_benci)))
        #计算活化容量【本次求解的航班所占的容量，现在需要将它们的容量解放出来】
        self.dict_capacity_huohua=copy.deepcopy(dict_empty_qiujie)
        for i in self.dict_xuhaodep0arr1_V_diedai_benci:#字典的key值
            self.dict_V_xuhaodep0arr1=self.dict_xuhaodep0arr1_V_diedai_benci[i][0]#xuhao-dep0/arr1
            self.dict_V_time15=self.dict_xuhaodep0arr1_V_diedai_benci[i][1]#15min时间片
            self.dict_V_day=self.dict_xuhaodep0arr1_V_diedai_benci[i][2]#天数
            self.dict_V_qingjing=self.dict_xuhaodep0arr1_V_diedai_benci[i][3]#情景
            self.dict_V_jichang=self.dict_xuhaodep0arr1_V_diedai_benci[i][4]#机场
            self.dict_capacity_huohua[self.dict_V_day][self.dict_V_jichang][self.dict_V_qingjing][self.dict_V_time15]+=1
        self.sum_huohua=0
        for i in self.dict_capacity_huohua:
            for j in self.dict_capacity_huohua[i]:
                for k in self.dict_capacity_huohua[i][j]:
                    for l in self.dict_capacity_huohua[i][j][k]:
                        self.sum_huohua+=self.dict_capacity_huohua[i][j][k][l]
        print('本次迭代最终活化的容量个数{}'.format(self.sum_huohua))
        #得到当前剩余的容量【初始容量减去迭代解】
        self.sum33=0
        for i in MODEL.dict_capacity_remain_diedai_benci:
            for j in MODEL.dict_capacity_remain_diedai_benci[i]:
                for k in MODEL.dict_capacity_remain_diedai_benci[i][j]:
                    for l in MODEL.dict_capacity_remain_diedai_benci[i][j][k]:
                        self.sum33+=MODEL.dict_capacity_remain_diedai_benci[i][j][k][l]
        #求得初始解后的剩余容量，数值可能一样，但内部容量组成可能不同
        print('本次迭代前剩余的容量个数{}'.format(self.sum33))
        #计算得到本次优化所使用的容量
        self.capacity_qiujie_benci=copy.deepcopy(dict_empty_qiujie)
        for i in self.capacity_qiujie_benci:
            for j in self.capacity_qiujie_benci[i]:
                for k in self.capacity_qiujie_benci[i][j]:
                    for l in self.capacity_qiujie_benci[i][j][k]:
                        self.capacity_qiujie_benci[i][j][k][l]=MODEL.dict_capacity_remain_diedai_benci[i][j][k][l]+self.dict_capacity_huohua[i][j][k][l]
                        #以防万一出现容量未负的问题
                        if self.capacity_qiujie_benci[i][j][k][l]<0:
                            self.capacity_qiujie_benci[i][j][k][l]=0
        self.sum_diedai_benci=0
        for i in self.capacity_qiujie_benci:
            for j in self.capacity_qiujie_benci[i]:
                for k in self.capacity_qiujie_benci[i][j]:
                    for l in self.capacity_qiujie_benci[i][j][k]:
                        self.sum_diedai_benci+=self.capacity_qiujie_benci[i][j][k][l]
        print('本次迭代时实际可用的容量个数{}'.format(self.sum_diedai_benci))
        print('------------------------------------------------------------------------------')
        print('------------------------------------------------------------------------------')

    def input_zhijieqiujie(self):
        self.a=self.input_total
        self.a=self.a.reset_index(drop=True)
        print('总共有{}个航班xuhao-dep0arr1'.format(len(self.a)))

    def presolve_data(self):
        #1决策变量
        #建立第一阶段决策变量索引的字典，用航班序号i来索引时间t
        self.dict_wit_jia1={}
        #建立a['xuhao-dep0/arr1']与a['Tw_it_jia1']
        for i in self.a.index.tolist():
            self.dict_wit_jia1[self.a['xuhao-dep0/arr1'][i]]=self.a['Tw_it_jia1'][i]
        ##print(len(self.dict_wit_jia1))
        #打印第一个索引的键值对
        # 转换为列表并打印第一个键值对
        ##print(list(self.dict_wit_jia1.items())[0])
        #建立第二阶段决策变量索引的字典，用航班序号i来索引天数d
        self.dict_wid_jia1={}
        #建立a['xuhao-dep0/arr1']与a['x_TIAN']
        for i in self.a.index.tolist():
            self.dict_wid_jia1[self.a['xuhao-dep0/arr1'][i]]=self.a['x_TIAN'][i]

        #2决策变量
        #建立第二阶段决策变量索引的字典，用航班序号i来索引时间t
        self.dict_xitds_it_jia1={}
        #建立a['xuhao-dep0/arr1']与a['Tx_itds']
        for i in self.a.index.tolist():
            self.dict_xitds_it_jia1[self.a['xuhao-dep0/arr1'][i]]=self.a['Tx_itds_jia1'][i]
        ##print(len(self.dict_xitds_it_jia1))
        ##print(list(self.dict_xitds_it_jia1.items())[0])
        #建立第二阶段决策变量索引的字典，用航班序号i来索引天数d
        self.dict_xitds_id={}
        #建立a['xuhao-dep0/arr1']与a['x_TIAN']
        for i in self.a.index.tolist():
            self.dict_xitds_id[self.a['xuhao-dep0/arr1'][i]]=self.a['x_TIAN'][i]
        ##print(len(self.dict_xitds_id))
        ##print(list(self.dict_xitds_id.items())[0])
        self.dict_xitds_iT={}
        #建立a['xuhao-dep0/arr1']与a['Tx_iTds']
        for i in self.a.index.tolist():
            self.dict_xitds_iT[self.a['xuhao-dep0/arr1'][i]]=self.a['Tx_iTds'][i]
        ##print(len(self.dict_xitds_iT))
        ##print(list(self.dict_xitds_iT.items())[0])

        #3约束con0、con1
        #约束0：索引t+1的航班i与t+1的对应字典
        self.dict_wit_zuihou1={}
        for i in self.a.index.tolist():
            self.dict_wit_zuihou1[self.a['xuhao-dep0/arr1'][i]]=[self.a['Tw_it_jia1'][i][-1]]
        ##print(len(self.dict_wit_zuihou1))
        ##print(list(self.dict_wit_zuihou1.items())[0])
        #约束1：索引航班i不加t+1的时刻
        self.dict_wit={}
        for i in self.a.index.tolist():
            self.dict_wit[self.a['xuhao-dep0/arr1'][i]]=self.a['Tw_it'][i]
        ##print(len(self.dict_wit))
        ##print(list(self.dict_wit.items())[0])

        #4约束con2、con3
        #约束2：建立各个航班i对应的Tfp
        self.dict_Tfp={}
        for i in self.a.index.tolist():
            self.dict_Tfp[self.a['xuhao-dep0/arr1'][i]]=self.a['Tfp'][i]
        ##print(len(self.dict_Tfp))
        ##print(list(self.dict_Tfp.items())[0:2])

        #5约束con4
        self.xuhao=list(set(self.a['xuhao']))
        #xuhao_dep为xuhao元素加上'-0'
        self.xuhao_dep = [str(x) + '-0' for x in self.xuhao]
        self.xuhao_arr = [str(x) + '-1' for x in self.xuhao]
        ##print(self.xuhao_dep[0])
        ##print(self.xuhao_arr[0])
        self.dict_flytime={}
        for i in self.a.index.tolist():
            self.dict_flytime[self.a['xuhao-dep0/arr1'][i]]=self.a['flytime'][i]
        ##print(list(self.dict_flytime.items())[0])
        self.dict_depday={}#考虑跨天航班，跨天要补288
        for i in self.a.index.tolist():
            self.dict_depday[self.a['xuhao-dep0/arr1'][i]]=self.a['TIAN_DEP'][i]
        ##print(list(self.dict_depday.items())[0])
        self.dict_arrday={}#考虑跨天航班，跨天要补288
        for i in self.a.index.tolist():
            self.dict_arrday[self.a['xuhao-dep0/arr1'][i]]=self.a['TIAN_ARR'][i]
        ##print(list(self.dict_arrday.items())[0])

        #6约束con5
        #不包含nan值
        self.liancheng=list(set(self.a['liancheng'].dropna()))
        #去掉两边的双引号""
        self.liancheng1=[i.split(',')[0][1:] for i in self.liancheng]
        self.liancheng1 = [s.replace("'", "") for s in self.liancheng1]
        self.liancheng2=[i.split(',')[1][1:-1] for i in self.liancheng]
        self.liancheng2 = [s.replace("'", "") for s in self.liancheng2]
        # ##print(liancheng)
        # ##print(liancheng1)
        # ##print(liancheng2)

        #7约束con6
        #建立第二阶段决策变量索引的字典，用航班序号i来索引时间t
        self.dict_xitds_it_zuihou1={}
        #建立a['xuhao-dep0/arr1']与a['Tx_itds']
        for i in self.a.index.tolist():
            self.dict_xitds_it_zuihou1[self.a['xuhao-dep0/arr1'][i]]=[self.a['Tx_itds_jia1'][i][-1]]
        ##print(len(self.dict_xitds_it_zuihou1))
        ##print(list(self.dict_xitds_it_zuihou1.items())[0])
        #约束6：索引航班i不加t+1的时刻
        self.dict_xitds={}
        for i in self.a.index.tolist():
            self.dict_xitds[self.a['xuhao-dep0/arr1'][i]]=self.a['Tx_itds'][i]
        ##print(len(self.dict_xitds))
        ##print(list(self.dict_xitds.items())[0])
        
        #8约束con7
        ###print(self.xuhao_dep)
        #9约束con8
        ###print(self.xuhao_arr)
        #10约束con9
        # ##print(liancheng)
        # ##print(liancheng1)
        # ##print(liancheng2)
        
        #11约束con10
        #设置最小、最大飞行时间
        self.dict_minflytime={}
        for i in self.a.index.tolist():
            self.dict_minflytime[self.a['xuhao-dep0/arr1'][i]]=self.a['minflytime'][i]
        ##print(list(self.dict_minflytime.items())[0:2])
        self.dict_maxflytime={}
        for i in self.a.index.tolist():
            self.dict_maxflytime[self.a['xuhao-dep0/arr1'][i]]=self.a['maxflytime'][i]
        ##print(list(self.dict_maxflytime.items())[0:2])
        
        #12约束con11
        #依据所选的航班计划，得到机场在每一天的起飞航班和降落航班
        self.airport51=self.A
        self.airport51_dep=[i + '-D' for i in self.airport51]
        self.airport51_arr=[i + '-A' for i in self.airport51]
        self.airport51_dep_arr=self.airport51_dep+self.airport51_arr#索引A机场
        ##print(self.airport51_dep_arr)
        #得到a中airport和xuhao-dep0/arr1列值的对应字典
        self.dict_airport_xuhaodep0arr1={}
        for i in self.a.index.tolist():
            self.dict_airport_xuhaodep0arr1[self.a['airport'][i]]=self.a['xuhao-dep0/arr1'][i]
        ##print(self.dict_airport_xuhaodep0arr1)
        #得到a中xuhao-dep0/arr1和x_TIAN列值的对应字典
        self.dict_xuhaodep0arr1_x_TIAN={}
        for i in self.a.index.tolist():
            self.dict_xuhaodep0arr1_x_TIAN[self.a['xuhao-dep0/arr1'][i]]=self.a['x_TIAN'][i]
        ##print(self.dict_xuhaodep0arr1_x_TIAN)
        
        #建立3索引字典，用机场、天数和15min时间片来索引航班序号，用机场、天数和15min时间片来索引航班序号
        self.list_a=set(self.a['xuhao-dep0/arr1'].tolist())#得到a的xuhao-dep0/arr1的列表
        self.dict_xuhao_dep0_arr1={}
        for i in self.dict_airport_day_15min_xuhaodep0arr1_total:
            self.dict_xuhao_dep0_arr1[i]={}
            for j in self.dict_airport_day_15min_xuhaodep0arr1_total[i]:
                self.dict_xuhao_dep0_arr1[i][j]={}
                for k in self.dict_airport_day_15min_xuhaodep0arr1_total[i][j]:
                    #xuhao_list为dict_airport_day_15min_xuhaodep0arr1_total[i][j][k]与list_a的共同元素
                    self.dict_xuhao_dep0_arr1[i][j][k]=list(set(self.dict_airport_day_15min_xuhaodep0arr1_total[i][j][k]).intersection(self.list_a))

        #第1阶段的航班计划运行天数dict，为目标函数第一阶段的航班时刻申请设置天数权重
        self.dict_xuhaodep0arr1_daylen={}
        for i in self.a.index.tolist():
            self.dict_xuhaodep0arr1_daylen[self.a.loc[i,'xuhao-dep0/arr1']] = self.a.loc[i,'day_len']
        ##print(list(self.dict_xuhaodep0arr1_daylen.items())[0])

    def add_variable(self):#[[[WPBA]]]
        #1、2决策变量
        #真正的决策变量的数据准备
        self.var_W_itd=gp.tuplelist([(i,t,d) for i in self.a['xuhao-dep0/arr1'] for t in self.dict_wit_jia1[i] for d in self.dict_wid_jia1[i]]) #第一阶段决策变量
        #print(len(self.var_W_itd),self.var_W_itd[0])
        self.var_P_id=gp.tuplelist([(i,d) for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]]) #目标函数的第一阶段间接决策变量
        #print(len(self.var_P_id),self.var_P_id[0])
        self.var_X_itds=gp.tuplelist([(i,t,d,s) for i in self.a['xuhao-dep0/arr1'] #第二阶段决策变量
                                for t in self.dict_xitds_it_jia1[i] for d in self.dict_xitds_id[i] for s in self.qingjing])
        #print(len(self.var_X_itds),self.var_X_itds[0])
        self.var_Z_ids=gp.tuplelist([(i,d,s) for i in self.a['xuhao-dep0/arr1'] #第二阶段间接决策变量【航班战术运行时间--15min的】
                                for d in self.dict_xitds_id[i] for s in self.qingjing])
        #print(len(self.var_Z_ids),self.var_Z_ids[0])
        self.var_V_iTds=gp.tuplelist([(i,t,d,s) for i in self.a['xuhao-dep0/arr1'] #第二阶段决策变量
                                for t in self.dict_xitds_iT[i] for d in self.dict_xitds_id[i] for s in self.qingjing])
        #print(len(self.var_V_iTds),self.var_V_iTds[0])
        self.var_Q_ids=gp.tuplelist([(i,d,s) for i in self.xuhao_arr #目标函数的间接决策变量【arr航班的战术延误指示变量，延误arr则为1，提前arr则为0】
                                for d in self.dict_xitds_id[i] for s in self.qingjing])
        #print(len(self.var_Q_ids),self.var_Q_ids[0])
        self.var_B_id=gp.tuplelist([(i,d) for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]]) #目标函数的第一阶段间接决策变量【表示xuhao-dep0/arr1的5min运行时间】
        #print(len(self.var_B_id),self.var_B_id[0])
        self.var_U_ids=gp.tuplelist([(i,d,s) for i in self.a['xuhao-dep0/arr1'] #第二阶段间接决策变量【航班战术运行时间--5min的】
                                for d in self.dict_xitds_id[i] for s in self.qingjing])
        #print(len(self.var_U_ids),self.var_U_ids[0])
        self.var_A_id=gp.tuplelist([(i,d) for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]]) #目标函数的第一阶段间接决策变量【表示xuhao-dep0/arr1的战略目标函数贡献】
        #print(len(self.var_A_i),self.var_A_i[0])
        self.var_Y_ids=gp.tuplelist([(i,d,s) for i in self.a['xuhao-dep0/arr1'] #第二阶段间接决策变量【在d天s情景的航班战术目标函数贡献】
                                for d in self.dict_xitds_id[i] for s in self.qingjing])
        #print(len(self.var_Y_ids),self.var_Y_ids[0])

        #1、2决策变量#[[[WPBA]]]
        #真正的决策变量
        self.W_itd=self.m.addVars(self.var_W_itd,lb=0,ub=1,vtype=gp.GRB.BINARY,name='W_itd')
        self.X_itds=self.m.addVars(self.var_X_itds,lb=0,ub=1,vtype=gp.GRB.BINARY,name='X_itds')
        self.Z_ids=self.m.addVars(self.var_Z_ids,lb=0,ub=95,vtype=gp.GRB.INTEGER,name='Z_ids')
        self.V_iTds=self.m.addVars(self.var_V_iTds,lb=0,ub=1,vtype=gp.GRB.BINARY,name='V_iTds')
        self.Q_ids=self.m.addVars(self.var_Q_ids,lb=0,ub=1,vtype=gp.GRB.BINARY,name='Q_ids')
        self.P_id=self.m.addVars(self.var_P_id,lb=-1,ub=1,vtype=gp.GRB.INTEGER,name='P_id')
        self.B_id=self.m.addVars(self.var_B_id,lb=0,ub=287,vtype=gp.GRB.INTEGER,name='B_id')
        self.U_ids=self.m.addVars(self.var_U_ids,lb=0,ub=287,vtype=gp.GRB.INTEGER,name='U_ids')
        self.A_id=self.m.addVars(self.var_A_id,lb=-3,ub=3,vtype=gp.GRB.INTEGER,name='A_id')
        self.Y_ids=self.m.addVars(self.var_Y_ids,lb=-287,ub=36,vtype=gp.GRB.INTEGER,name='Y_ids')

    def add_constraints12(self):
        #约束1,t+1时刻的决策变量设置为0，它是用来debug的
        self.m.addConstrs((self.W_itd[i,t,d]==0 for i in self.a['xuhao-dep0/arr1'] for t in self.dict_wit_zuihou1[i] for d in self.dict_wid_jia1[i]),name='con0')
        #约束2,第一阶段决策变量不减性约束
        self.m.addConstrs((self.W_itd[i,t,d]>=self.W_itd[i,t+1,d] for i in self.a['xuhao-dep0/arr1'] for t in self.dict_wit[i] for d in self.dict_wid_jia1[i]),name='con1')
        #print('约束1、2设置完毕')

        #约束3-,xuhao-dep0/arr1的战略起飞时间【间接决策变量】
        self.m.addConstrs((gp.quicksum(t*(self.W_itd[i,t,d]-self.W_itd[i,t+1,d]) for t in self.dict_wit[i])
                           ==self.B_id[i,d] for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]),name='con2-')
        #print('约束3-设置完毕')

        #约束3,所有dep/arr航班最大时刻偏移设置,前移后移均是15min,即3个时间片【在0.对照组中，偏移量为0】
        self.m.addConstrs(((self.B_id[i,d]-self.dict_Tfp[i])>=-3 for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]),name='con2')
        self.m.addConstrs(((self.B_id[i,d]-self.dict_Tfp[i])<=3 for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]),name='con2+')
        #print('约束3设置完毕')

        #补充约束3+，确保航班申请一定被分配时刻，不会全为0
        self.m.addConstrs((gp.quicksum(self.W_itd[i,t,d] for t in self.dict_wit[i])>=1 for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]),name='con3+')
        #print('约束3+设置完毕')

        #约束4，战略阶段航班的飞行时间不变
        self.m.addConstrs((self.B_id[i2,d2]-self.B_id[i1,d1] +288*(d2-d1) == self.dict_flytime[i1]
                    for i1,i2 in zip(self.xuhao_dep,self.xuhao_arr) for d2,d1 in zip(self.dict_arrday[i2],self.dict_depday[i1])),name='con4')
        #print('约束4设置完毕')

        #约束5，战略阶段联程航班的最小周转时间限制，都是同日联程航班
        self.m.addConstrs((self.B_id[i2,d2]-self.B_id[i1,d1] +288*(d2-d1)>= 6
                    for i1,i2 in zip(self.liancheng1,self.liancheng2) for d2,d1 in zip(self.dict_depday[i2],self.dict_arrday[i1])),name='con5')
        #print('约束5设置完毕')

        #约束5+,xuhao-dep0/arr1的战略时刻偏移【间接决策变量】
        self.m.addConstrs((self.B_id[i,d]-self.dict_Tfp[i]==self.A_id[i,d] for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]),name='con5+')
        #print('约束5+设置完毕')

        #约束6,t+1时刻的决策变量设置为0，它是用来debug的
        self.m.addConstrs((self.X_itds[i,t,d,s]==0 
                    for i in self.a['xuhao-dep0/arr1'] 
                    for t in self.dict_xitds_it_zuihou1[i] 
                    for d in self.dict_xitds_id[i] 
                    for s in self.qingjing),name='con6')
        #约束7,第一阶段决策变量不减性约束
        self.m.addConstrs((self.X_itds[i,t,d,s]>=self.X_itds[i,t+1,d,s] 
                    for i in self.a['xuhao-dep0/arr1'] 
                    for t in self.dict_xitds[i] 
                    for d in self.dict_xitds_id[i] 
                    for s in self.qingjing),name='con7')
        #print('约束6、7设置完毕')

        #约束7+，xuhao-dep0/arr1的战术起飞时间【间接决策变量】
        self.m.addConstrs((gp.quicksum(t*(self.X_itds[i,t,d,s]-self.X_itds[i,t+1,d,s]) for t in self.dict_xitds[i])==self.U_ids[i,d,s] 
                    for i in self.a['xuhao-dep0/arr1'] for d in self.dict_xitds_id[i] for s in self.qingjing),name='con7+')

        #约束8：战术阶段dep航班的延误值限制，至少为0且最大为36
        self.m.addConstrs((self.U_ids[i,d,s]-self.B_id[i,d] >= 0 
                    for i in self.xuhao_dep for d in self.dict_xitds_id[i] for s in self.qingjing),name='con8')
        self.m.addConstrs((self.U_ids[i,d,s]-self.B_id[i,d] <= 36
                    for i in self.xuhao_dep for d in self.dict_xitds_id[i] for s in self.qingjing),name='con9')
        #print('约束8设置完毕')

        #约束9：战术阶段arr航班的延误值限制，最大为36
        self.m.addConstrs((self.U_ids[i,d,s]-self.B_id[i,d] <= 36
                    for i in self.xuhao_arr for d in self.dict_xitds_id[i] for s in self.qingjing),name='con10')
        #print('约束9设置完毕')
        
        #约束9-：战术阶段航班的延误量，为负时表示提前，为正时表示延误
        self.m.addConstrs((self.U_ids[i,d,s]-self.B_id[i,d] == self.Y_ids[i,d,s]
                    for i in self.xuhao_dep for d in self.dict_xitds_id[i] for s in self.qingjing),name='con10-')
        #print('约束9-设置完毕')
        
        #约束9+：战术阶段航班的延误量，为负时表示提前，为正时表示延误
        self.m.addConstrs((self.U_ids[i,d,s]-self.B_id[i,d] == self.Y_ids[i,d,s]
                    for i in self.xuhao_arr for d in self.dict_xitds_id[i] for s in self.qingjing),name='con10+')
        #print('约束9+设置完毕')
        
        #补充约束9'，确保航班申请一定被分配时刻，不会全为0
        self.m.addConstrs((gp.quicksum(self.X_itds[i,t,d,s] for t in self.dict_xitds[i])>=1 
                    for i in self.a['xuhao-dep0/arr1'] 
                    for d in self.dict_xitds_id[i] 
                    for s in self.qingjing),name='con9+')
        #print('约束9+设置完毕')

        #约束10：战术阶段不同天的联程航班的最小周转时间限制
        self.m.addConstrs((self.U_ids[i2,d2,s]-self.U_ids[i1,d1,s] +288*(d2-d1)>= 6
                    for i1,i2 in zip(self.liancheng1,self.liancheng2) for d1,d2 in zip(self.dict_xitds_id[i1],self.dict_xitds_id[i2]) for s in self.qingjing),name='con11')
        #print('约束10设置完毕')

        #约束11：战术阶段dep/arr航班对的最小和最大飞行时间限制
        self.m.addConstrs((self.U_ids[i2,d2,s]-self.U_ids[i1,d1,s]+288*(d2-d1)>= self.dict_minflytime[i1]
                    for i1,i2 in zip(self.xuhao_dep,self.xuhao_arr) for d1,d2 in zip(self.dict_xitds_id[i1],self.dict_xitds_id[i2]) for s in self.qingjing),name='con12')

        self.m.addConstrs((self.U_ids[i2,d2,s]-self.U_ids[i1,d1,s]+288*(d2-d1)<= self.dict_maxflytime[i1]
                    for i1,i2 in zip(self.xuhao_dep,self.xuhao_arr) for d1,d2 in zip(self.dict_xitds_id[i1],self.dict_xitds_id[i2]) for s in self.qingjing),name='con13')
        #print('约束11设置完毕')

        #约束12：战术阶段的机场容量约束的指示决策变量约束【Z_ids是15min航班战术运行时刻】
        self.m.addConstrs((self.U_ids[i,d,s]/3-self.Z_ids[i,d,s]<=0.999
                    for i in self.a['xuhao-dep0/arr1'] for d in self.dict_xitds_id[i] for s in self.qingjing),name='con14')
        self.m.addConstrs((self.U_ids[i,d,s]/3-self.Z_ids[i,d,s]>=0
                    for i in self.a['xuhao-dep0/arr1'] for d in self.dict_xitds_id[i] for s in self.qingjing),name='con15')

        #战术阶段的机场容量约束的指示决策变量约束【V_iTds是15min航班战术运行时刻】
        #决策变量V_iTds的唯一性约束
        self.m.addConstrs((gp.quicksum(self.V_iTds[i,t,d,s] for t in self.dict_xitds_iT[i])==1 
                    for i in self.a['xuhao-dep0/arr1'] 
                    for d in self.dict_xitds_id[i] 
                    for s in self.qingjing),name='con16')

        #决策变量V_iTds与Z_ids的关系约束
        self.m.addConstrs((gp.quicksum(t*self.V_iTds[i,t,d,s] for t in self.dict_xitds_iT[i])==self.Z_ids[i,d,s] 
                    for i in self.a['xuhao-dep0/arr1'] 
                    for d in self.dict_xitds_id[i] 
                    for s in self.qingjing),name='con17')

        #基于V_iTds的容量约束【有航班的D、air、n才设置容量约束】
        for s in self.qingjing: 
            for n in range(0, 96):#这里的n是取了0-95
                for d in range(150, 153): #这里的d是取了150-153
                    for air in self.airport51_dep_arr: 
                        hangbanlist=self.dict_xuhao_dep0_arr1[air][d][n]#这里筛选出的航班i在d天、n时间片上均有决策变量的定义
                        #如果hangbanlist为空，则跳过
                        if len(hangbanlist)==0:
                            continue
                        else:
                            self.m.addConstr((gp.quicksum(self.V_iTds[i,n,d,s] for i in hangbanlist) 
                                        <= self.capacity_qiujie_benci[d][air][s][n])
                                        ,name='{},{},{},{}'.format(d, air, s, n))
        #print('约束12设置完毕')

        #约束13：目标函数需要使用的指示决策变量Q_ids[i,d,s]、P_i[i]
        self.m.addConstrs((self.P_id[i,d]*self.P_id[i,d]==1 
                    for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]),name='con18')

        self.m.addConstrs((self.P_id[i,d]*(self.B_id[i,d]-self.dict_Tfp[i])>=0 
                    for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i]),name='con19')

        self.m.addConstrs((self.Q_ids[i,d,s]*(self.U_ids[i,d,s]-self.B_id[i,d])>=0 
                    for i in self.xuhao_arr for d in self.dict_xitds_id[i] for s in self.qingjing),name='con20')

        self.m.addConstrs(((1-self.Q_ids[i,d,s])*(self.U_ids[i,d,s]-self.B_id[i,d])<=0 
                    for i in self.xuhao_arr for d in self.dict_xitds_id[i] for s in self.qingjing),name='con21')
        #print('约束13设置完毕')

    def set_objective1(self):
        #添加目标函数
        self.m.setObjective((gp.quicksum((self.B_id[i,d]-self.dict_Tfp[i])*self.P_id[i,d] 
                                                 for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i])
                        +gp.quicksum(1*self.qingjing_weight[s]*(self.U_ids[i,d,s]-self.B_id[i,d]) 
                                             for i in self.xuhao_dep for d in self.dict_xitds_id[i] for s in self.qingjing) 
                        +gp.quicksum(1*self.qingjing_weight[s]*self.Q_ids[i,d,s]*(self.U_ids[i,d,s]-self.B_id[i,d]) 
                                             for i in self.xuhao_arr for d in self.dict_xitds_id[i] for s in self.qingjing)),
                            gp.GRB.MINIMIZE)
        #print('目标函数设置完毕')

    def set_objective2(self):#带有机场层面拥挤重要度的两阶段
        #添加目标函数
        self.m.setObjective((gp.quicksum((self.B_id[i,d]-self.dict_Tfp[i])*self.P_id[i,d]*self.dict_xuhao_delay_importance[i] 
                                                 for i in self.a['xuhao-dep0/arr1'] for d in self.dict_wid_jia1[i])
                        +gp.quicksum(self.dict_xuhao_delay_importance[i]*self.qingjing_weight[s]*(self.U_ids[i,d,s]-self.B_id[i,d]) 
                                             for i in self.xuhao_dep for d in self.dict_xitds_id[i] for s in self.qingjing) 
                        +gp.quicksum(self.dict_xuhao_delay_importance[i]*self.qingjing_weight[s]*self.Q_ids[i,d,s]*(self.U_ids[i,d,s]-self.B_id[i,d]) 
                                             for i in self.xuhao_arr for d in self.dict_xitds_id[i] for s in self.qingjing)),
                            gp.GRB.MINIMIZE)
        #print('目标函数设置完毕')

    def optimize_qiujie1(self):#默认设置还是可以的，调整之后效果也不太好
        self.m.Params.TimeLimit = 600
        self.m.Params.MIPGap = 0.01
        self.m.Params.SolutionLimit = 35
        self.m.update()
        self.m.optimize()
        self.time_chushi=self.m.Runtime

    def optimize_diedai1(self):#默认设置还是可以的，调整之后效果也不太好
        self.m.Params.TimeLimit = 600
        self.m.update()
        self.m.optimize()
        self.time_diedai=self.m.Runtime

    def optimize_zhijie1(self):#默认设置还是可以的，调整之后效果也不太好
        self.m.Params.MIPGap = 0.003
        self.m.update()
        self.m.optimize()

    def get_solution_chushi1(self):
        self.m.write('../../output/two-stage/1/chushijie/{}.sol'.format(self.III))
        MODEL.chushi_qingkuang.loc[self.III]=[self.III,self.time_chushi]
        MODEL.chushi_qingkuang.to_csv('../../output/two-stage/chushixinxi/chushi_qingkuang{}.csv'.format(self.III),index=False)

    def get_solution_diedai1(self):
        #如果有解，就输出sol文件
        if self.m.Status==2:#有解且最优
            self.m.write('../../output/two-stage/1/diedaijie/{}.sol'.format(self.III))
            #执行函数update_diedai_youjie
            self.update_diedai_youjie_zuiyou()
            self.baocun_diedai1()
        elif self.m.SolCount==0:
            #达到Timelimit，且无解
            self.update_diedai_wujie()
            self.baocun_diedai1()
        elif self.m.Status==3:
            #问题无解
            self.update_diedai_wujie()
            self.baocun_diedai1()
        else:#有解但不是最优
            self.m.write('../../output/two-stage/1/diedaijie/{}.sol'.format(self.III))
            self.update_diedai_youjie()
            self.baocun_diedai1()

    def baocun_diedai1(self):
        #将MODEL.time_window_weight0变为dataframe，并输出为csv
        # 将字典的键转换为字符串
        self.time_window_weight0_str = {str(key): value for key, value in MODEL.time_window_weight0.items()}
        self.time_window_weight_benci_df=pd.DataFrame(list(self.time_window_weight0_str.items()),columns=['time_window','weight'])
        self.time_window_weight_benci_df.to_csv('../../output/two-stage/time_window_weight/yuan{}.csv'.format(self.III),index=False)

        #将MODEL.dict_time_window_bili0变为dataframe，并输出为csv
        # 将字典的键转换为字符串
        self.dict_time_window_bili0_str = {str(key): value for key, value in MODEL.time_window_bili.items()}
        self.dict_time_window_bili_benci_df=pd.DataFrame(list(self.dict_time_window_bili0_str.items()),columns=['time_window','bili'])
        self.dict_time_window_bili_benci_df.to_csv('../../output/two-stage/time_window_bili/yuan{}.csv'.format(self.III),index=False)

        #将MODEL.dict_xuhao_weight0变为dataframe，并输出为csv
        # 将字典的键转换为字符串
        self.dict_xuhao_weight0_str = {str(key): value for key, value in MODEL.dict_xuhao_weight0.items()}
        self.dict_xuhao_weight_benci_df=pd.DataFrame(list(self.dict_xuhao_weight0_str.items()),columns=['dict_xuhao','weight'])
        self.dict_xuhao_weight_benci_df.to_csv('../../output/two-stage/xuhao_weight/yuan{}.csv'.format(self.III),index=False)

        #将MODEL.dict_xuhao_obj0变为dataframe，并输出为csv
        # 将字典的键转换为字符串
        self.dict_xuhao_obj0_str = {str(key): value for key, value in MODEL.dict_xuhao_obj0.items()}
        self.dict_xuhao_obj_benci_df=pd.DataFrame(list(self.dict_xuhao_obj0_str.items()),columns=['dict_xuhao','obj'])
        self.dict_xuhao_obj_benci_df.to_csv('../../output/two-stage/xuhao_obj/yuan{}.csv'.format(self.III),index=False)

        self.list_time15=[i for i in range(0,96)]
        self.num=0
        for i in [150,151,152]:
            for j in [0, 1, 2]:
                self.variable_name = "c{}_{}".format(i, j)
                globals()[self.variable_name] = pd.DataFrame(index=self.airport51_dep_arr, columns=self.list_time15)
                for k in self.airport51_dep_arr:
                    for h in self.list_time15:
                        globals()[self.variable_name].loc[k,h]=MODEL.dict_capacity_remain_diedai_benci[i][k][j][h]
                        self.num+=MODEL.dict_capacity_remain_diedai_benci[i][k][j][h]
                globals()[self.variable_name].to_csv('../../output/two-stage/capacity_remain_diedai_benci/c{}_{}.csv'.format(i,j),index=False)
        print('第{}次迭代保存的剩余容量为{}'.format(self.III,self.num))
        #输出字典MODEL.dict_capacity_remain_diedai_benci
        import json
        # # 假设你的字典为 MODEL.dict_capacity_remain_diedai_benci
        # # 将字典保存为JSON文件
        # self.file_path = '../../output/two-stage/capacity_remain_diedai_benci/yuan.json'#文件路径
        # with open(self.file_path, "w") as self.outfile:
        #     json.dump(MODEL.dict_capacity_remain_diedai_benci, self.outfile)
        # # 指定保存字典的JSON文件路径
        # self.file_path = '../../output/two-stage/capacity_remain_diedai_benci/yuan.json'
        # # 从JSON文件中读取字典
        # with open(self.file_path, "r") as self.infile:
        #     MODEL.dict_capacity_remain_diedai_benci_df = json.load(self.infile)
        # 假设你的字典为 MODEL.dict_xuhaodep0arr1_V_suoyou
        # 将字典保存为JSON文件
        self.file_path = '../../output/two-stage/xuhaodep0arr1_V_suoyou/yuan.json'#文件路径
        with open(self.file_path, "w") as self.outfile:
            json.dump(MODEL.dict_xuhaodep0arr1_V_suoyou, self.outfile)

        #将MODEL.dict_xuhaodep0arr1_objzl0变为dataframe，并输出为csv
        # 将字典的键转换为字符串
        self.dict_xuhaodep0arr1_objzl0_str = {str(key): value for key, value in MODEL.dict_xuhaodep0arr1_objzl0.items()}
        self.dict_xuhaodep0arr1_objzl_benci_df=pd.DataFrame(list(self.dict_xuhaodep0arr1_objzl0_str.items()),columns=['xuhao-dep0/arr1','zlobj'])
        self.dict_xuhaodep0arr1_objzl_benci_df.to_csv('../../output/two-stage/diedaimubiaozl/yuan{}.csv'.format(self.III),index=False)
        #将MODEL.dict_xuhaodep0arr1_objzs0变为dataframe，并输出为csv
        # 将字典的键转换为字符串
        self.dict_xuhaodep0arr1_objzs0_str = {str(key): value for key, value in MODEL.dict_xuhaodep0arr1_objzs0.items()}
        self.dict_xuhaodep0arr1_objzs_benci_df=pd.DataFrame(list(self.dict_xuhaodep0arr1_objzs0_str.items()),columns=['xuhao-dep0/arr1','zsobj'])
        self.dict_xuhaodep0arr1_objzs_benci_df.to_csv('../../output/two-stage/diedaimubiaozs/yuan{}.csv'.format(self.III),index=False)

    def get_solution_zhijie1(self):#默认设置还是可以的，调整之后效果也不太好
        self.m.write('../../output/two-stage/1/zhijiejie/zhijie1.sol')

    #@classmethod#类方法,不需要实例化就可以调用【一般都会有cls这个参数】
    def update_capacity_used_qiujie1(self):
        #得到.sol中V_iTds为1的值，即航班i在d天情景s下的战术运行时刻T15min
        self.V=[]
        for i in self.m.getVars():
            if i.varname[0]=='V' and i.x==1:
                self.V.append(i)
    
        #提取V中元素对应的航班序号、时间片、天数、情景和机场信息到字典dict_V
        self.dict_V={}
        for i in range(0,len(self.V)):
            self.air=self.dict_xuhao_jichang[self.V[i].varname.split(',')[0][7:]]
            #如果air不在airport51_dep_arr中，则跳过
            if self.air not in self.airport51_dep_arr:
                continue
            else:
                self.dict_V[i]=[self.V[i].varname.split(',')[0][7:],float(self.V[i].varname.split(',')[1]),
                                float(self.V[i].varname.split(',')[2]),float(self.V[i].varname.split(',')[3][:-1]),self.air]
    
        #根据本次求解的结果，更新类变量capacity_used_qiujie，获得历史累计的容量使用情况
        self.sum4=0
        for i in self.dict_V:
            self.dict_V_xuhaodep0arr1=self.dict_V[i][0]#xuhao-dep0/arr1
            self.dict_V_time15=self.dict_V[i][1]#15min时间片
            self.dict_V_day=self.dict_V[i][2]#天数
            self.dict_V_qingjing=self.dict_V[i][3]#情景
            self.dict_V_jichang=self.dict_V[i][4]#机场
            MODEL.capacity_used_qiujie[self.dict_V_day][self.dict_V_jichang][self.dict_V_qingjing][self.dict_V_time15]+=1
            self.sum4+=1
        print('第{}次求解,那45个机场添加了{}个航班的容量使用情况'.format(self.III,self.sum4))
        #得到目标函数值
        self.objective=self.m.objVal
        MODEL.objective+=self.objective#+=是累加，=+是赋值
        print('第{}次求解,目标函数总值为{}'.format(self.III,MODEL.objective))
        print('---------------------------------------------------------------------')
        print('---------------------------------------------------------------------')

    def update_diedai_youjie_zuiyou(self):
        #从gorubi求解器内得到的目标函数值
        self.objective=self.m.objVal
        print('从gurobi求解器中得到目标函数值{}'.format(self.objective))
        
        #从.sol文件中计算本次迭代xuhao的目标函数值                                      ###
        self.dict_xuhaodep0arr1_obj_benci={}#这是本次迭代的所有xuhao-dep0/arr1的目标函数值
        self.sum00=0# 统计战术延误
        self.sum11=0# 统计战术提前【这部分没有算在延误里，归为0了】
        self.sum22=0# 统计战略目标函数值
        self.dict_xuhaodep0arr1_zlobj_benci={}#这是本次迭代的所有xuhao-dep0/arr1的战略目标函数值
        #读入本次计算出来的.sol文件
        with open('../../output/two-stage/1/diedaijie/{}.sol'.format(self.III), "r") as f:
            f.seek(0)#重新定义文件指针【每次只能读取一次】
            for line in f:
                if line[0]=='A':
                    self.dict_xuhaodep0arr1_zlobj_benci[line.split(' ')[0].split('[')[1][:-1]]=float(line.split(' ')[1])
        #求出战略目标函数值之和
        for i in self.dict_xuhaodep0arr1_zlobj_benci:
            self.sum22+=abs(self.dict_xuhaodep0arr1_zlobj_benci[i])
        print('本次迭代解的战略目标函数值为{}'.format(self.sum22))
        #转化为航班号索引字典
        
        self.keys=list(self.dict_xuhaodep0arr1_zlobj_benci.keys())
        self.keys1=[]
        self.dict_xuhaodep0arr1_zlobj_benci1={}
        for l in self.keys:
            self.keys1.append(l.split(',')[0])
        self.keys1=list(set(self.keys1))
        for l in self.keys1:
            self.ll=[]
            for k in self.keys:
                if k.split(',')[0]==l:
                    self.ll.append(self.dict_xuhaodep0arr1_zlobj_benci[k])
                    self.llsum=0
                    for j in self.ll:
                        self.llsum+=abs(j)
            self.dict_xuhaodep0arr1_zlobj_benci1[l]=self.llsum
        # print(dict_xuhaodep0arr1_zlobj1)
        self.summmmm=0
        for i in self.dict_xuhaodep0arr1_zlobj_benci1:
            self.summmmm+=abs(self.dict_xuhaodep0arr1_zlobj_benci1[i])
        self.summmmmm=0
        for i in self.dict_xuhaodep0arr1_zlobj_benci:
            self.summmmmm+=abs(self.dict_xuhaodep0arr1_zlobj_benci[i])
        print('本次迭代解的战略目标函数值为{},{}'.format(self.summmmm,self.summmmmm))
        self.dict_xuhaodep0arr1_zlobj_benci=self.dict_xuhaodep0arr1_zlobj_benci1
        
        #制作xuhao-dep0arr1和战术目标值的对应关系                                      ###
        self.dict_xuhaodep0arr1_zsobj_benci={}
        self.dict_jianjie_benci={}
        #读入本次计算出来的.sol文件
        with open('../../output/two-stage/1/diedaijie/{}.sol'.format(self.III), "r") as f:
            f.seek(0)
            for line in f:
                if line[0]=='Y':
                    #先将所有的战术延误目标函数值的决策变量放入字典
                    self.dict_jianjie_benci[tuple([line.split(' ')[0].split(',')[0][6:],line.split(' ')[0].split(',')[1],line.split(' ')[0].split(',')[2][:-1]])]=float(line.split(' ')[1])
        #开始计算每个xuhao-dep0arr1的战术目标函数值
        self.xuhao_input=self.a['xuhao-dep0/arr1'].tolist()
        for xuhaoi in self.xuhao_input:
            self.zs_obj=0
            for i in self.dict_jianjie_benci:
                if i[0]==xuhaoi:
                    if self.dict_jianjie_benci[i]>0:#只加战术延误的，提前的不加
                        self.zs_obj+=self.dict_jianjie_benci[i]*self.qingjing_weight[int(i[2])]
            self.dict_xuhaodep0arr1_zsobj_benci[xuhaoi]=self.zs_obj
        #这里得到本次迭代的战术延误、提前的目标函数值之和
        for i in self.dict_jianjie_benci:
            if self.dict_jianjie_benci[i]>0:
                self.sum00+=self.dict_jianjie_benci[i]*self.qingjing_weight[int(i[2])]
            else:
                self.sum11+=self.dict_jianjie_benci[i]*self.qingjing_weight[int(i[2])]
        print('本次迭代解的战术目标函数延误值为{}'.format(self.sum00))
        print('本次迭代解的战术目标函数提前值为{}'.format(self.sum11))
        #得到xuhao-dep0arr1的总体目标函数值：包含战略、战术两部分
        for i in self.dict_xuhaodep0arr1_zlobj_benci:
            self.dict_xuhaodep0arr1_obj_benci[i]=abs(self.dict_xuhaodep0arr1_zlobj_benci[i])+self.dict_xuhaodep0arr1_zsobj_benci[i]
        #这就是本次优化得到的xuhao的权重，也就是目标函数值
        self.dict_xuhao_weight_benci={}
        for i in list(set(self.a['xuhao'].tolist())):
            self.o=0
            #得到input1中的xuhao对应的xuhao-dep0/arr1列值的列表
            self.list_xuhao_dep0arr1=self.a[self.a['xuhao']==i]['xuhao-dep0/arr1'].tolist()
            for j in self.list_xuhao_dep0arr1:
                self.o+=self.dict_xuhaodep0arr1_obj_benci[j]#都是正的
            self.dict_xuhao_weight_benci[i]=self.o
        #计算依据本次迭代的.sol文件得到的目标函数总值
        self.objective_benci=0
        for i in self.dict_xuhao_weight_benci:
            self.objective_benci+=self.dict_xuhao_weight_benci[i]
        print('从.sol文件中得到目标函数值{}'.format(self.objective_benci))
        
        #【历史目标函数值，用于比较】得到self.xuhao_select中所有xuhao的优化前的目标函数值总和
        self.objective_xuhao_select=0
        for i in self.xuhao_select:
            self.objective_xuhao_select+=MODEL.dict_xuhao_obj0[i]#【类变量：调用】
        print('这些xuhao的历史目标函数值{}'.format(self.objective_xuhao_select))
        print('---------------------------------------------------------------------')

        #上面在计算迭代前和迭代后的目标函数值的变化
        #如果结果更优了，要更新筛选参数并改变解
        if self.objective_benci<self.objective_xuhao_select:
            #则更新类变量dict_xuhao_weight0，更新xuhao的权重以及xuhao的目标函数值
            for i in list(set(self.a['xuhao'].tolist())):
                MODEL.dict_xuhao_obj0[i]=self.dict_xuhao_weight_benci[i]#【类变量：变更】
                MODEL.dict_xuhao_weight0[i]=0#最优解的时候，将权重全部置为0，它们无法优化了，但依然可能因最大关联操作被选中【类变量：变更】
            #更新类变量time_window_weight0，更新时间窗的权重【由于其本身有更优的可能，所以该时间窗的权重不下降】
            self.weight_TW=MODEL.time_window_weight0[self.time_window_weight_max]#得到本次时间窗的权重
            MODEL.time_window_weight0[self.time_window_weight_max]=MODEL.time_window_weight0[self.time_window_weight_max]*math.exp(-1)
            self.weight_TW_biangeng=MODEL.time_window_weight0[self.time_window_weight_max]#得到迭代变化后的时间窗的权重
            #更新时刻选择bili【由于最优了，所以计算量不密集，该时间窗的bili上升】
            self.bili_TW=MODEL.time_window_bili[self.time_window_weight_max]#得到本次时间窗的xuhao选择bili
            MODEL.time_window_bili[self.time_window_weight_max]=MODEL.time_window_bili[self.time_window_weight_max]*1.2
            self.bili_TW_biangeng=MODEL.time_window_bili[self.time_window_weight_max]#得到迭代变化后的时间窗xuhao选择bili
            #此时有更优解，xuhao-dep0arr1的战略、战术目标函数值迭代
            for i in self.dict_xuhaodep0arr1_zlobj_benci:
                MODEL.dict_xuhaodep0arr1_objzl0[i]=self.dict_xuhaodep0arr1_zlobj_benci[i]#【类变量：变更】【有负的】
            for i in self.dict_xuhaodep0arr1_zsobj_benci:
                MODEL.dict_xuhaodep0arr1_objzs0[i]=self.dict_xuhaodep0arr1_zsobj_benci[i]#【类变量：变更】
            
            #更新当前解的V_iTds【先删除再导入】
            print('未变更前的dict_V_iTds的长度{}'.format(len(MODEL.dict_xuhaodep0arr1_V_suoyou)))
            self.llll=self.key_huohua#这都是在air45_dep_arr中的航班，且一定存在于dict_xuhaodep0arr1_V中的
            print('dict_V_iTds要更新的(暂时删除的)长度{}'.format(len(self.llll)))
            #将dict_xuhaodep0arr1_V中key为llll内元素的键值对删除，用来存储新的迭代V_iTds
            for i in self.llll:
                del MODEL.dict_xuhaodep0arr1_V_suoyou[i]#【类变量：变更】
            print('变更后的dict_V_iTds的长度{}'.format(len(MODEL.dict_xuhaodep0arr1_V_suoyou)))
            #计算本次迭代xuhao的V结果字典                               ###
            self.dict_xuhaodep0arr1_V_benci={}
            self.num=0
            with open('../../output/two-stage/1/diedaijie/{}.sol'.format(self.III), "r") as f:
                f.seek(0)
                for line in f:
                    if line[0]=='V' and line.split(' ')[1][0]=='1':
                        self.xh=line.split(' ')[0].split(',')[0][7:]
                        self.V=[self.xh,float(line.split(' ')[0].split(',')[1]),float(line.split(' ')[0].split(',')[2]),
                            float(line.split(' ')[0].split(',')[3][0]),self.dict_xuhaodep0arr1_airport[self.xh]]
                        self.dict_xuhaodep0arr1_V_benci[str(self.num)+'-{}'.format(self.III)]=self.V
                        self.num+=1
            print('本次.sol文件得到的V_iTds==1的个数有{}'.format(len(self.dict_xuhaodep0arr1_V_benci)))
            # 创建一个空列表来存储要删除的键【不在45个机场之中】
            self.keys_to_delete = []
            #将机场不在airport51_dep_arr中的item删除
            for i in self.dict_xuhaodep0arr1_V_benci:
                if self.dict_xuhaodep0arr1_V_benci[i][4] not in self.airport51_dep_arr:
                    self.keys_to_delete.append(i)
            for key in self.keys_to_delete:
                self.dict_xuhaodep0arr1_V_benci.pop(key)
            print('本次添加更新的dict_V_iTds的长度{}'.format(len(self.dict_xuhaodep0arr1_V_benci)))
            #将dict_xuhaodep0arr1_V_benci中的元素添加到dict_xuhaodep0arr1_V_suoyou中
            for i in self.dict_xuhaodep0arr1_V_benci:
                MODEL.dict_xuhaodep0arr1_V_suoyou[i]=self.dict_xuhaodep0arr1_V_benci[i]
            print('本次添加后的dict_V_iTds的长度{}'.format(len(MODEL.dict_xuhaodep0arr1_V_suoyou)))
            
            #本次迭代已使用的容量                         ###
            self.dict_used_diedai_benci=copy.deepcopy(dict_empty_qiujie)
            for i in self.dict_xuhaodep0arr1_V_benci:#字典的key值
                self.dict_V_xuhaodep0arr1=self.dict_xuhaodep0arr1_V_benci[i][0]#xuhao-dep0/arr1
                self.dict_V_time15=self.dict_xuhaodep0arr1_V_benci[i][1]#15min时间片
                self.dict_V_day=self.dict_xuhaodep0arr1_V_benci[i][2]#天数
                self.dict_V_qingjing=self.dict_xuhaodep0arr1_V_benci[i][3]#情景
                self.dict_V_jichang=self.dict_xuhaodep0arr1_V_benci[i][4]#机场
                self.dict_used_diedai_benci[self.dict_V_day][self.dict_V_jichang][self.dict_V_qingjing][self.dict_V_time15]+=1
            self.sum=0
            for i in self.dict_used_diedai_benci:
                for j in self.dict_used_diedai_benci[i]:
                    for k in self.dict_used_diedai_benci[i][j]:
                        for l in self.dict_used_diedai_benci[i][j][k]:
                            self.sum+=self.dict_used_diedai_benci[i][j][k][l]
            print('本次迭代已使用的容量{}'.format(self.sum))
            #计算本次迭代的剩余容量,并更新类变量capacity_remain_diedai_benci
            MODEL.dict_capacity_remain_diedai_benci=copy.deepcopy(dict_empty_qiujie)
            self.sum=0
            for i in MODEL.dict_capacity_remain_diedai_benci:
                for j in MODEL.dict_capacity_remain_diedai_benci[i]:
                    for k in MODEL.dict_capacity_remain_diedai_benci[i][j]:
                        for l in MODEL.dict_capacity_remain_diedai_benci[i][j][k]:
                            MODEL.dict_capacity_remain_diedai_benci[i][j][k][l]=self.capacity_qiujie_benci[i][j][k][l]-self.dict_used_diedai_benci[i][j][k][l]
                            self.sum+=MODEL.dict_capacity_remain_diedai_benci[i][j][k][l]
            print('本次迭代后剩余的容量{}'.format(self.sum))
            
            #记录diedai_qingkuang
            self.ob=0#记录本次迭代优化后的整体目标函数值
            for i in MODEL.dict_xuhao_obj0:
                self.ob+=MODEL.dict_xuhao_obj0[i]
            self.ob_zl=0#记录本次迭代优化后的战略目标函数值
            for i in MODEL.dict_xuhaodep0arr1_objzl0:
                self.ob_zl+=abs(MODEL.dict_xuhaodep0arr1_objzl0[i])
            self.ob_zs=0#记录本次迭代优化后的战术目标函数值
            for i in MODEL.dict_xuhaodep0arr1_objzs0:
                self.ob_zs+=MODEL.dict_xuhaodep0arr1_objzs0[i]
            MODEL.diedai_qingkuang.loc[self.III]=[self.III,self.time_window_weight_max,self.ob,self.ob_zl,self.ob_zs,1,1,0,self.bili_TW,
                                                  self.bili_TW_biangeng,self.weight_TW,self.weight_TW_biangeng,1,self.time_diedai]
            MODEL.diedai_qingkuang.to_csv('../../output/two-stage/diedaixinxi/diedai_qingkuang{}.csv'.format(self.III),index=False)
            
            print('------------------------------------------------------------------------------')
            print('------------------------------------------------------------------------------')
            #预计下次时间窗选择和bili
            #依据dict_time_window_weight生成dict_time_window_weight的value值最大的时间窗序号
            self.time_window_weight_max=sorted(MODEL.time_window_weight0.items(),key=lambda x:x[1],reverse=True)[0][0]#【类字典】
            print('预计下次迭代选择的时间窗为{}号时间窗'.format(self.time_window_weight_max))
            #得到本次时间窗的时刻选择比例
            self.bili=MODEL.time_window_bili[self.time_window_weight_max]#【类字典】
            print('预计下次迭代选择的时间窗xuhao比例为{}'.format(self.bili))
            print('*******************************************************************************')

        else:#是当前最优解，但结果并不更优，只更新参数
            #则更新类变量dict_xuhao_weight0，更新xuhao的权重
            for i in list(set(self.a['xuhao'].tolist())):
                MODEL.dict_xuhao_weight0[i]=0#最优解的时候，将xuhao权重全部置为0，但并不代表它们不会被选到
            #更新类变量time_window_weight0，更新时间窗的权重【由于其本身没有更优的可能，所以该时间窗的权重下降】
            self.weight_TW=MODEL.time_window_weight0[self.time_window_weight_max]
            MODEL.time_window_weight0[self.time_window_weight_max]=(MODEL.time_window_weight0[self.time_window_weight_max])*math.exp(-1)*(1-MODEL.time_window_bili[self.time_window_weight_max])
            self.weight_TW_biangeng=MODEL.time_window_weight0[self.time_window_weight_max]
            #更新时刻选择bili【由于最优了，所以计算量不密集，该时间窗的bili上升】
            self.bili_TW=MODEL.time_window_bili[self.time_window_weight_max]
            MODEL.time_window_bili[self.time_window_weight_max]=MODEL.time_window_bili[self.time_window_weight_max]*1.2
            self.bili_TW_biangeng=MODEL.time_window_bili[self.time_window_weight_max]
            
            #解没有更新，所以容量剩余和V这两个类变量就不用变化，直接记录diedai_qingkuang
            self.ob=0#记录本次迭代优化后的整体目标函数值
            for i in MODEL.dict_xuhao_obj0:
                self.ob+=MODEL.dict_xuhao_obj0[i]
            self.ob_zl=0#记录本次迭代优化后的战略目标函数值
            for i in MODEL.dict_xuhaodep0arr1_objzl0:
                self.ob_zl+=abs(MODEL.dict_xuhaodep0arr1_objzl0[i])
            self.ob_zs=0#记录本次迭代优化后的战术目标函数值
            for i in MODEL.dict_xuhaodep0arr1_objzs0:
                self.ob_zs+=MODEL.dict_xuhaodep0arr1_objzs0[i]
            MODEL.diedai_qingkuang.loc[self.III]=[self.III,self.time_window_weight_max,self.ob,self.ob_zl,self.ob_zs,1,1,0,self.bili_TW,
                                                  self.bili_TW_biangeng,self.weight_TW,self.weight_TW_biangeng,0,self.time_diedai]
            MODEL.diedai_qingkuang.to_csv('../../output/two-stage/diedaixinxi/diedai_qingkuang{}.csv'.format(self.III),index=False)
            
            print('------------------------------------------------------------------------------')
            print('------------------------------------------------------------------------------')
            #预计下次时间窗选择和bili
            #依据dict_time_window_weight生成dict_time_window_weight的value值最大的时间窗序号
            self.time_window_weight_max=sorted(MODEL.time_window_weight0.items(),key=lambda x:x[1],reverse=True)[0][0]#【类字典】
            print('预计下次迭代选择的时间窗为{}号时间窗'.format(self.time_window_weight_max))
            #得到本次时间窗的时刻选择比例
            self.bili=MODEL.time_window_bili[self.time_window_weight_max]#【类字典】
            print('预计下次迭代选择的时间窗xuhao比例为{}'.format(self.bili))
            print('*******************************************************************************')

    def update_diedai_youjie(self):
        #从gorubi求解器内得到的目标函数值
        self.objective=self.m.objVal
        print('从gurobi求解器中得到目标函数值{}'.format(self.objective))
        
        #从.sol文件中计算本次迭代xuhao的目标函数值                                      ###
        self.dict_xuhaodep0arr1_obj_benci={}#这是本次迭代的所有xuhao-dep0/arr1的目标函数值
        self.sum00=0# 统计战术延误
        self.sum11=0# 统计战术提前【这部分没有算在延误里，归为0了】
        self.sum22=0# 统计战略目标函数值
        self.dict_xuhaodep0arr1_zlobj_benci={}#这是本次迭代的所有xuhao-dep0/arr1的战略目标函数值
        #读入本次计算出来的.sol文件
        with open('../../output/two-stage/1/diedaijie/{}.sol'.format(self.III), "r") as f:
            f.seek(0)#重新定义文件指针【每次只能读取一次】
            for line in f:
                if line[0]=='A':
                    self.dict_xuhaodep0arr1_zlobj_benci[line.split(' ')[0].split('[')[1][:-1]]=float(line.split(' ')[1])
        #求出战略目标函数值之和
        for i in self.dict_xuhaodep0arr1_zlobj_benci:
            self.sum22+=abs(self.dict_xuhaodep0arr1_zlobj_benci[i])
        print('本次迭代解的战略目标函数值为{}'.format(self.sum22))
        #转化为航班号索引字典
        
        self.keys=list(self.dict_xuhaodep0arr1_zlobj_benci.keys())
        self.keys1=[]
        self.dict_xuhaodep0arr1_zlobj_benci1={}
        for l in self.keys:
            self.keys1.append(l.split(',')[0])
        self.keys1=list(set(self.keys1))
        for l in self.keys1:
            self.ll=[]
            for k in self.keys:
                if k.split(',')[0]==l:
                    self.ll.append(self.dict_xuhaodep0arr1_zlobj_benci[k])
                    self.llsum=0
                    for j in self.ll:
                        self.llsum+=abs(j)
            self.dict_xuhaodep0arr1_zlobj_benci1[l]=self.llsum
        # print(dict_xuhaodep0arr1_zlobj1)
        self.summmmm=0
        for i in self.dict_xuhaodep0arr1_zlobj_benci1:
            self.summmmm+=abs(self.dict_xuhaodep0arr1_zlobj_benci1[i])
        self.summmmmm=0
        for i in self.dict_xuhaodep0arr1_zlobj_benci:
            self.summmmmm+=abs(self.dict_xuhaodep0arr1_zlobj_benci[i])
        print('本次迭代解的战略目标函数值为{},{}'.format(self.summmmm,self.summmmmm))
        self.dict_xuhaodep0arr1_zlobj_benci=self.dict_xuhaodep0arr1_zlobj_benci1
        
        #制作xuhao-dep0arr1和战术目标值的对应关系                                      ###
        self.dict_xuhaodep0arr1_zsobj_benci={}
        self.dict_jianjie_benci={}
        #读入本次计算出来的.sol文件
        with open('../../output/two-stage/1/diedaijie/{}.sol'.format(self.III), "r") as f:
            f.seek(0)
            for line in f:
                if line[0]=='Y':
                    #先将所有的战术延误目标函数值的决策变量放入字典
                    self.dict_jianjie_benci[tuple([line.split(' ')[0].split(',')[0][6:],line.split(' ')[0].split(',')[1],line.split(' ')[0].split(',')[2][:-1]])]=float(line.split(' ')[1])
        #开始计算每个xuhao-dep0arr1的战术目标函数值
        self.xuhao_input=self.a['xuhao-dep0/arr1'].tolist()
        for xuhaoi in self.xuhao_input:
            self.zs_obj=0
            for i in self.dict_jianjie_benci:
                if i[0]==xuhaoi:
                    if self.dict_jianjie_benci[i]>0:#只加战术延误的，提前的不加
                        self.zs_obj+=self.dict_jianjie_benci[i]*self.qingjing_weight[int(i[2])]
            self.dict_xuhaodep0arr1_zsobj_benci[xuhaoi]=self.zs_obj
        #这里得到本次迭代的战术延误、提前的目标函数值之和
        for i in self.dict_jianjie_benci:
            if self.dict_jianjie_benci[i]>0:
                self.sum00+=self.dict_jianjie_benci[i]*self.qingjing_weight[int(i[2])]
            else:
                self.sum11+=self.dict_jianjie_benci[i]*self.qingjing_weight[int(i[2])]
        print('本次迭代解的战术目标函数延误值为{}'.format(self.sum00))
        print('本次迭代解的战术目标函数提前值为{}'.format(self.sum11))
        #得到xuhao-dep0arr1的总体目标函数值：包含战略、战术两部分
        for i in self.dict_xuhaodep0arr1_zlobj_benci:
            self.dict_xuhaodep0arr1_obj_benci[i]=abs(self.dict_xuhaodep0arr1_zlobj_benci[i])+self.dict_xuhaodep0arr1_zsobj_benci[i]
        #这就是本次优化得到的xuhao的权重，也就是目标函数值
        self.dict_xuhao_weight_benci={}
        for i in list(set(self.a['xuhao'].tolist())):
            self.o=0
            #得到input1中的xuhao对应的xuhao-dep0/arr1列值的列表
            self.list_xuhao_dep0arr1=self.a[self.a['xuhao']==i]['xuhao-dep0/arr1'].tolist()
            for j in self.list_xuhao_dep0arr1:
                self.o+=self.dict_xuhaodep0arr1_obj_benci[j]
            self.dict_xuhao_weight_benci[i]=self.o
        #计算依据本次迭代的.sol文件得到的目标函数总值
        self.objective_benci=0
        for i in self.dict_xuhao_weight_benci:
            self.objective_benci+=self.dict_xuhao_weight_benci[i]
        print('从.sol文件中得到目标函数值{}'.format(self.objective_benci))
        
        #【历史目标函数值，用于比较】得到self.xuhao_select中所有xuhao的优化前的目标函数值总和
        self.objective_xuhao_select=0
        for i in self.xuhao_select:
            self.objective_xuhao_select+=MODEL.dict_xuhao_obj0[i]#【类变量：调用】
        print('这些xuhao的历史目标函数值{}'.format(self.objective_xuhao_select))


        #上面在计算迭代前和迭代后的目标函数值的变化
        #如果结果更优了，要更新筛选参数并改变解
        if self.objective_benci<self.objective_xuhao_select:
            #则更新类变量dict_xuhao_weight0，更新xuhao的权重以及xuhao的目标函数值
            for i in list(set(self.a['xuhao'].tolist())):
                MODEL.dict_xuhao_obj0[i]=self.dict_xuhao_weight_benci[i]#【类变量：变更】
                MODEL.dict_xuhao_weight0[i]=self.dict_xuhao_weight_benci[i]#因为值更优了，但未最优，仍可优化，所以这些xuhao就更新为新的目标值
            #更新类变量time_window_weight0，更新时间窗的权重【由于其本身有更优的可能，所以该时间窗的权重不下降】
            self.weight_TW=MODEL.time_window_weight0[self.time_window_weight_max]#得到本次时间窗的权重
            MODEL.time_window_weight0[self.time_window_weight_max]=MODEL.time_window_weight0[self.time_window_weight_max]*math.exp(-1)#【类变量：变更】
            self.weight_TW_biangeng=MODEL.time_window_weight0[self.time_window_weight_max]#得到迭代变化后的时间窗的权重
            #更新时刻选择bili【由于更优了，但没有最优，计算量较密集，该时间窗的bili下降】
            self.bili_TW=MODEL.time_window_bili[self.time_window_weight_max]
            MODEL.time_window_bili[self.time_window_weight_max]=MODEL.time_window_bili[self.time_window_weight_max]*0.9
            self.bili_TW_biangeng=MODEL.time_window_bili[self.time_window_weight_max]#得到迭代变化后的时间窗xuhao选择bili
            #此时有更优解，xuhao-dep0arr1的战略、战术目标函数值迭代
            for i in self.dict_xuhaodep0arr1_zlobj_benci:
                MODEL.dict_xuhaodep0arr1_objzl0[i]=self.dict_xuhaodep0arr1_zlobj_benci[i]#【类变量：变更】
            for i in self.dict_xuhaodep0arr1_zsobj_benci:
                MODEL.dict_xuhaodep0arr1_objzs0[i]=self.dict_xuhaodep0arr1_zsobj_benci[i]#【类变量：变更】
            
            #更新当前解的V_iTds【先删除再导入】
            print('未变更前的dict_V_iTds的长度{}'.format(len(MODEL.dict_xuhaodep0arr1_V_suoyou)))
            self.llll=self.key_huohua#这都是在air45_dep_arr中的航班，且一定存在于dict_xuhaodep0arr1_V中的
            print('dict_V_iTds要更新的(暂时删除的)长度{}'.format(len(self.llll)))
            #将dict_xuhaodep0arr1_V中key为llll内元素的键值对删除，用来存储新的迭代V_iTds
            for i in self.llll:
                del MODEL.dict_xuhaodep0arr1_V_suoyou[i]#【类变量：变更】
            print('变更后的dict_V_iTds的长度{}'.format(len(MODEL.dict_xuhaodep0arr1_V_suoyou)))
            #计算本次迭代xuhao的V结果字典                               ###
            self.dict_xuhaodep0arr1_V_benci={}
            self.num=0
            with open('../../output/two-stage/1/diedaijie/{}.sol'.format(self.III), "r") as f:
                f.seek(0)#重新定义文件指针【每次只能读取一次】
                for line in f:
                    if line[0]=='V' and line.split(' ')[1][0]=='1':
                        self.xh=line.split(' ')[0].split(',')[0][7:]
                        self.V=[self.xh,float(line.split(' ')[0].split(',')[1]),float(line.split(' ')[0].split(',')[2]),
                            float(line.split(' ')[0].split(',')[3][0]),self.dict_xuhaodep0arr1_airport[self.xh]]
                        self.dict_xuhaodep0arr1_V_benci[str(self.num)+'-{}'.format(self.III)]=self.V#【num变量要记得加self】
                        self.num+=1
            print('本次.sol文件得到的V_iTds==1的个数有{}'.format(len(self.dict_xuhaodep0arr1_V_benci)))
            # 创建一个空列表来存储要删除的键【不在45个机场之中】
            self.keys_to_delete = []
            #将机场不在airport51_dep_arr中的item删除
            for i in self.dict_xuhaodep0arr1_V_benci:
                if self.dict_xuhaodep0arr1_V_benci[i][4] not in self.airport51_dep_arr:
                    self.keys_to_delete.append(i)
            for key in self.keys_to_delete:
                self.dict_xuhaodep0arr1_V_benci.pop(key)
            print('本次添加更新的dict_V_iTds的长度{}'.format(len(self.dict_xuhaodep0arr1_V_benci)))
            #将dict_xuhaodep0arr1_V_benci中的元素添加到dict_xuhaodep0arr1_V_suoyou中
            for i in self.dict_xuhaodep0arr1_V_benci:
                MODEL.dict_xuhaodep0arr1_V_suoyou[i]=self.dict_xuhaodep0arr1_V_benci[i]
            print('本次添加后的dict_V_iTds的长度{}'.format(len(MODEL.dict_xuhaodep0arr1_V_suoyou)))
            
            #本次迭代已使用的容量                         ###
            self.dict_used_diedai_benci=copy.deepcopy(dict_empty_qiujie)
            for i in self.dict_xuhaodep0arr1_V_benci:#字典的key值
                self.dict_V_xuhaodep0arr1=self.dict_xuhaodep0arr1_V_benci[i][0]#xuhao-dep0/arr1
                self.dict_V_time15=self.dict_xuhaodep0arr1_V_benci[i][1]#15min时间片
                self.dict_V_day=self.dict_xuhaodep0arr1_V_benci[i][2]#天数
                self.dict_V_qingjing=self.dict_xuhaodep0arr1_V_benci[i][3]#情景
                self.dict_V_jichang=self.dict_xuhaodep0arr1_V_benci[i][4]#机场
                self.dict_used_diedai_benci[self.dict_V_day][self.dict_V_jichang][self.dict_V_qingjing][self.dict_V_time15]+=1
            self.sum=0
            for i in self.dict_used_diedai_benci:
                for j in self.dict_used_diedai_benci[i]:
                    for k in self.dict_used_diedai_benci[i][j]:
                        for l in self.dict_used_diedai_benci[i][j][k]:
                            self.sum+=self.dict_used_diedai_benci[i][j][k][l]
            print('本次迭代已使用的容量{}'.format(self.sum))
            #计算本次迭代的剩余容量,并更新类变量capacity_remain_diedai_benci
            MODEL.dict_capacity_remain_diedai_benci=copy.deepcopy(dict_empty_qiujie)
            self.sum=0
            for i in MODEL.dict_capacity_remain_diedai_benci:
                for j in MODEL.dict_capacity_remain_diedai_benci[i]:
                    for k in MODEL.dict_capacity_remain_diedai_benci[i][j]:
                        for l in MODEL.dict_capacity_remain_diedai_benci[i][j][k]:
                            MODEL.dict_capacity_remain_diedai_benci[i][j][k][l]=self.capacity_qiujie_benci[i][j][k][l]-self.dict_used_diedai_benci[i][j][k][l]
                            self.sum+=MODEL.dict_capacity_remain_diedai_benci[i][j][k][l]
            print('本次迭代后剩余的容量{}'.format(self.sum))
            
            #记录diedai_qingkuang
            self.ob=0#记录本次迭代优化后的整体目标函数值
            for i in MODEL.dict_xuhao_obj0:
                self.ob+=MODEL.dict_xuhao_obj0[i]
            self.ob_zl=0#记录本次迭代优化后的战略目标函数值
            for i in MODEL.dict_xuhaodep0arr1_objzl0:
                self.ob_zl+=abs(MODEL.dict_xuhaodep0arr1_objzl0[i])
            self.ob_zs=0#记录本次迭代优化后的战术目标函数值
            for i in MODEL.dict_xuhaodep0arr1_objzs0:
                self.ob_zs+=MODEL.dict_xuhaodep0arr1_objzs0[i]
            MODEL.diedai_qingkuang.loc[self.III]=[self.III,self.time_window_weight_max,self.ob,self.ob_zl,self.ob_zs,0,1,0,self.bili_TW,
                                                  self.bili_TW_biangeng,self.weight_TW,self.weight_TW_biangeng,1,self.time_diedai]
            MODEL.diedai_qingkuang.to_csv('../../output/two-stage/diedaixinxi/diedai_qingkuang{}.csv'.format(self.III),index=False)
            
            print('------------------------------------------------------------------------------')
            print('------------------------------------------------------------------------------')
            #预计下次时间窗选择和bili
            #依据dict_time_window_weight生成dict_time_window_weight的value值最大的时间窗序号
            self.time_window_weight_max=sorted(MODEL.time_window_weight0.items(),key=lambda x:x[1],reverse=True)[0][0]#【类字典】
            print('预计下次迭代选择的时间窗为{}号时间窗'.format(self.time_window_weight_max))
            #得到本次时间窗的时刻选择比例
            self.bili=MODEL.time_window_bili[self.time_window_weight_max]#【类字典】
            print('预计下次迭代选择的时间窗xuhao比例为{}'.format(self.bili))
            print('*******************************************************************************')
            

        else:#是当前可行解，但结果并不更优，只更新参数
            #则更新类变量dict_xuhao_weight0，更新xuhao的权重
            for i in list(set(self.a['xuhao'].tolist())):
                MODEL.dict_xuhao_weight0[i]=MODEL.dict_xuhao_weight0[i]*0.8#这些xuhao没有更优解，所以权重下降
            #更新类变量time_window_weight0，更新时间窗的权重【由于其本身没有达到更优，所以该时间窗的权重下降】
            self.weight_TW=MODEL.time_window_weight0[self.time_window_weight_max]
            MODEL.time_window_weight0[self.time_window_weight_max]=(MODEL.time_window_weight0[self.time_window_weight_max])*math.exp(-1)*(1-MODEL.time_window_bili[self.time_window_weight_max])
            self.weight_TW_biangeng=MODEL.time_window_weight0[self.time_window_weight_max]
            #更新时刻选择bili【由于未达到最优，所以计算量密集，该时间窗的bili下降】
            self.bili_TW=MODEL.time_window_bili[self.time_window_weight_max]
            MODEL.time_window_bili[self.time_window_weight_max]=MODEL.time_window_bili[self.time_window_weight_max]*0.9
            self.bili_TW_biangeng=MODEL.time_window_bili[self.time_window_weight_max]
            
            #解没有更新，所以容量剩余和V这两个类变量就不用变化，直接记录diedai_qingkuang
            self.ob=0#记录本次迭代优化后的整体目标函数值
            for i in MODEL.dict_xuhao_obj0:
                self.ob+=MODEL.dict_xuhao_obj0[i]
            self.ob_zl=0#记录本次迭代优化后的战略目标函数值
            for i in MODEL.dict_xuhaodep0arr1_objzl0:
                self.ob_zl+=abs(MODEL.dict_xuhaodep0arr1_objzl0[i])
            self.ob_zs=0#记录本次迭代优化后的战术目标函数值
            for i in MODEL.dict_xuhaodep0arr1_objzs0:
                self.ob_zs+=MODEL.dict_xuhaodep0arr1_objzs0[i]
            MODEL.diedai_qingkuang.loc[self.III]=[self.III,self.time_window_weight_max,self.ob,self.ob_zl,self.ob_zs,0,1,0,self.bili_TW,
                                                  self.bili_TW_biangeng,self.weight_TW,self.weight_TW_biangeng,0,self.time_diedai]
            MODEL.diedai_qingkuang.to_csv('../../output/two-stage/diedaixinxi/diedai_qingkuang{}.csv'.format(self.III),index=False)
    
            print('------------------------------------------------------------------------------')
            print('------------------------------------------------------------------------------')
            #预计下次时间窗选择和bili
            #依据dict_time_window_weight生成dict_time_window_weight的value值最大的时间窗序号
            self.time_window_weight_max=sorted(MODEL.time_window_weight0.items(),key=lambda x:x[1],reverse=True)[0][0]#【类字典】
            print('预计下次迭代选择的时间窗为{}号时间窗'.format(self.time_window_weight_max))
            #得到本次时间窗的时刻选择比例
            self.bili=MODEL.time_window_bili[self.time_window_weight_max]#【类字典】
            print('预计下次迭代选择的时间窗xuhao比例为{}'.format(self.bili))
            print('*******************************************************************************')

    def update_diedai_wujie(self):
        #则更新类变量dict_xuhao_weight0，更新xuhao的权重
        for i in list(set(self.a['xuhao'].tolist())):
            MODEL.dict_xuhao_weight0[i]=MODEL.dict_xuhao_weight0[i]*0.8#这些xuhao没有更优解，所以权重下降
        #更新类变量time_window_weight0，更新时间窗的权重【由于其本身没有达到更优，所以该时间窗的权重下降】
        self.weight_TW=MODEL.time_window_weight0[self.time_window_weight_max]
        MODEL.time_window_weight0[self.time_window_weight_max]=(MODEL.time_window_weight0[self.time_window_weight_max])*math.exp(-1)*(1-MODEL.time_window_bili[self.time_window_weight_max])
        self.weight_TW_biangeng=MODEL.time_window_weight0[self.time_window_weight_max]
        #更新时刻选择bili【由于根本无解，所以计算量密集，该时间窗的bili下降】
        self.bili_TW=MODEL.time_window_bili[self.time_window_weight_max]
        MODEL.time_window_bili[self.time_window_weight_max]=MODEL.time_window_bili[self.time_window_weight_max]*0.9
        self.bili_TW_biangeng=MODEL.time_window_bili[self.time_window_weight_max]

        #解没有更新，所以容量剩余和V这两个类变量就不用变化，直接记录diedai_qingkuang
        self.ob=0#记录本次迭代优化后的整体目标函数值
        for i in MODEL.dict_xuhao_obj0:
            self.ob+=MODEL.dict_xuhao_obj0[i]
        self.ob_zl=0#记录本次迭代优化后的战略目标函数值
        for i in MODEL.dict_xuhaodep0arr1_objzl0:
            self.ob_zl+=abs(MODEL.dict_xuhaodep0arr1_objzl0[i])
        self.ob_zs=0#记录本次迭代优化后的战术目标函数值
        for i in MODEL.dict_xuhaodep0arr1_objzs0:
            self.ob_zs+=MODEL.dict_xuhaodep0arr1_objzs0[i]
        MODEL.diedai_qingkuang.loc[self.III]=[self.III,self.time_window_weight_max,self.ob,self.ob_zl,self.ob_zs,0,0,1,self.bili_TW,
                                                self.bili_TW_biangeng,self.weight_TW,self.weight_TW_biangeng,0,self.time_diedai]
        MODEL.diedai_qingkuang.to_csv('../../output/two-stage/diedaixinxi/diedai_qingkuang{}.csv'.format(self.III),index=False)

        print('------------------------------------------------------------------------------')
        print('------------------------------------------------------------------------------')
        #预计下次时间窗选择和bili
        #依据dict_time_window_weight生成dict_time_window_weight的value值最大的时间窗序号
        self.time_window_weight_max=sorted(MODEL.time_window_weight0.items(),key=lambda x:x[1],reverse=True)[0][0]#【类字典】
        print('预计下次迭代选择的时间窗为{}号时间窗'.format(self.time_window_weight_max))
        #得到本次时间窗的时刻选择比例
        self.bili=MODEL.time_window_bili[self.time_window_weight_max]#【类字典】
        print('预计下次迭代选择的时间窗xuhao比例为{}'.format(self.bili))
        print('*******************************************************************************')


#### 求初始解

In [ ]:
# for III in range(97,97):
#         M=MODEL(III,input,A,capacity_initial,qingjing,qingjing_weight,dict_xuhao_jichang,dict_xuhao_delay_importance,
#             dict_airport_day_15min_xuhaodep0arr1_total,dict_zuhao_liuliangzhi,dict_sum,dict_time_window_xuhao,
#             dict_max_relation_xuhao,input_total,airport51_dep_arr,dict_xuhaodep0arr1_airport,dict_xuhao_tianshu)
#         M.input_qiujie()
#         M.update_qiujie_benci_capacity()
#         M.presolve_data()
#         M.add_variable()
#         M.add_constraints12()
#         M.set_objective1()#不考虑机场拥挤重要度
#         #M.set_objective2()#机场拥挤重要度
#         M.optimize_qiujie1()
#         M.get_solution_chushi1()
#         M.update_capacity_used_qiujie1()


#### 求迭代解

In [ ]:
for III in range(0,1000):
    M=MODEL(III,input,A,capacity_initial,qingjing,qingjing_weight,dict_xuhao_jichang,dict_xuhao_delay_importance,
            dict_airport_day_15min_xuhaodep0arr1_total,dict_zuhao_liuliangzhi,dict_sum,dict_time_window_xuhao,
            dict_max_relation_xuhao,input_total,airport51_dep_arr,dict_xuhaodep0arr1_airport,dict_xuhao_tianshu)
    M.input_diedai()
    M.update_diedai_benci_capacity()
    M.presolve_data()
    M.add_variable()
    M.add_constraints12()
    M.set_objective1()
    #M.set_objective2()
    M.optimize_diedai1()
    M.get_solution_diedai1()


#### 直接求解

In [ ]:
# III=0
# M=MODEL(III,input,A,capacity_initial,qingjing,qingjing_weight,dict_xuhao_jichang,dict_xuhao_delay_importance,
#                  dict_airport_day_15min_xuhaodep0arr1_total,dict_zuhao_liuliangzhi,dict_sum,dict_time_window_xuhao,dict_max_relation_xuhao,
#                  input_total,airport51_dep_arr,dict_xuhaodep0arr1_airport,dict_xuhao_tianshu)
# M.input_zhijieqiujie()
# M.update_qiujie_benci_capacity()
# M.presolve_data()
# M.add_variable()
# M.add_constraints12()
# M.set_objective1()#不考虑重要度
# M.optimize_zhijie1()
# M.get_solution_zhijie1()
